In [1]:
from ETGEMs_function_PMI import *
import pandas as pd
import cobra
import ast
from cobra.io import write_sbml_model
from numpy import *
import copy
import math

In [2]:
#Get Reaction G0 from local file_original
reaction_g0_file_original0 = './reaction_g0_strain0.txt'
reaction_g0_file_original1 = './reaction_g0_strain1.txt'
#Get Metabolite concentration from local file
metabolites_lnC_file_original0 = './metabolites_lnC_strain0.txt'
metabolites_lnC_file_original1 = './metabolites_lnC_strain1.txt'
#Get Model from local file
model_file_original0 = './iML1515.xml'
model_file_original1 = './iMM904.xml'
#Get reaction kcat data from ECMpy
reaction_kcat_MW_file_original0 = './ID_kcat_MW_file_strain0.csv'
reaction_kcat_MW_file_original1 = './ID_kcat_MW_file_strain1.csv'

In [3]:
## Convert to usable model formats
model0=Get_Concretemodel_Need_Data(reaction_g0_file_original0,metabolites_lnC_file_original0,model_file_original0,reaction_kcat_MW_file_original0)
model1=Get_Concretemodel_Need_Data(reaction_g0_file_original1,metabolites_lnC_file_original1,model_file_original1,reaction_kcat_MW_file_original1)

In [4]:
# get the information of km, vmax and public metabolites
km = pd.read_csv('./km.csv')
vmax = pd.read_csv('./vmax.csv')
public_metabolism = pd.read_csv('./public_metabolism_test.csv')

'''
public_metabolism_name_list = []
public_metabolism_concentration_list = []
for i in public_metabolism['metabolism']:
    public_metabolism_name_list.append(i)
for j in public_metabolism['concentration']:
    public_metabolism_concentration_list.append(j)
public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
print(public_metabolism)
'''

"\npublic_metabolism_name_list = []\npublic_metabolism_concentration_list = []\nfor i in public_metabolism['metabolism']:\n    public_metabolism_name_list.append(i)\nfor j in public_metabolism['concentration']:\n    public_metabolism_concentration_list.append(j)\npublic_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))\nprint(public_metabolism)\n"

In [5]:
# definate the function to simulate metabolism and interaction
def time_space_state(model_list, biomass_list, growth_list, breed_list, parameter_list, public_metabolism, vmax, km, deta_t, deta_x, micro_distribute_threshold, length, D, o2_concentration):
    
    number_cell_side = length // deta_x
    number_cell_side = int(number_cell_side)
    public_metabolism_name_list = []
    public_metabolism_concentration_list = []
    for i in public_metabolism['metabolism']:
        public_metabolism_name_list.append(i)
    for j in public_metabolism['concentration']:
        public_metabolism_concentration_list.append(j)
    public_metabolism = dict(zip(public_metabolism_name_list, public_metabolism_concentration_list))
    
    number_model = len(model_list)
    
    k_m = {}
    v_max = {}
    for i in range(number_model):
        for j in range(len(km['reactions_strain'+str(i)])):
            k_m[(i, km['reactions_strain'+str(i)][j])] = km['km_strain'+str(i)][j]
            v_max[(i, vmax['reactions_strain'+str(i)][j])] = vmax['vmax_strain'+str(i)][j]
    v_max_o2 = v_max[(0,'EX_glc__D_e_reverse')]
    k_m_o2 = k_m[(0,'EX_glc__D_e_reverse')]        
    
    number_public_metabolism = len(public_metabolism)
    distribute_micro_list = {}
    distribute_public_metabolism_list = {}
    distribute_lb_list = {}
    public_metabolism_r_list = []
    #set the initial distribution of straints
    for i in range(number_model):
        distribute_micro = np.random.randint(20, size=number_cell_side)
        distribute_micro_list.update({i: distribute_micro+1})
    print(distribute_micro_list)
    distribute_micro_list_initial = copy.deepcopy(distribute_micro_list)
    #set the initial distribution of metabolites
    for i in public_metabolism_name_list:
        distribute_public_metabolism = multiply(np.mat(np.ones(number_cell_side)), public_metabolism[i])
        distribute_public_metabolism_list.update({i: distribute_public_metabolism})
    #calculate the upperbounds of uptake reactions for public metabolites
    public_reaction_ub_list = {}
    public_reaction_list = {}
    for i in range(number_model):
        public_reaction_ub = {}
        public_reaction = []
        for rea in model_list[i]['reaction_list']:
            if 'EX_' in rea:
                for j in public_metabolism_name_list:
                    try:
                        model_list[i]['coef_matrix'][(j,rea)]
                    except:
                        pass
                    else:
                        ub = np.mat(np.ones(number_cell_side))
                        if model_list[i]['coef_matrix'][(j,rea)] > 0:
                            for m in range(number_cell_side):
                                ub[0,m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
                        else:
                            try:
                                model_list[i]['ub_list'][rea]
                            except:
                                ub = ub * 1000
                            else:
                                ub = ub * model_list[i]['ub_list'][rea]
                        public_reaction_ub.update({rea: ub})
                        public_reaction.append(rea)
                        break          
            public_reaction_ub_list[i] = public_reaction_ub
            public_reaction_list[i] = public_reaction

            
    ct = 0
    
    distribute_micro_list_t = {ct: distribute_micro_list}
    distribute_public_metabolism_list_t = {ct: distribute_public_metabolism_list}
    distribute_lb_list_t = {ct: distribute_lb_list}
    r = micro_distribute_threshold + 1
    r_threshold = [r]*5
    slip_r = np.mean(r_threshold[-5:])
    number = {}
    various = {}
    for i in range(number_model):
        number[i] = [np.mean(distribute_micro_list[i])]
        various[i] = [np.std(distribute_micro_list[i])/np.mean(distribute_micro_list[i])]
    
            
    # iterative simulation by slip_r
    biomass = {}
    while slip_r >= micro_distribute_threshold:
        ct += 1
        print(ct)
        distribute_micro_list_t[ct] = copy.deepcopy(distribute_micro_list_t[ct-1])
        micro_decrease = {}
        micro_increase = {}
        
        #simulate the cell wandering
        #micro_decrease: the decrease number of strains
        #micro_increase: the increase number of strains
        #micro_decrease_cell: the decrease number of strains in the current grid
        #micro_increase_fcell: the increase number of strains in the forward grid
        #micro_increase_bcell: the increase number of strains in the backward grid
        for m in range(number_cell_side):
            micro_decrease_cell = {}
            micro_increase_cell = {}
            for i in range(number_model):
                micro_decrease_cell[i] = 0
                micro_increase_cell[i] = 0
            micro_decrease[m] = micro_decrease_cell
            micro_increase[m] = micro_increase_cell
            
            
        if ct > 0:
            met = 'glc__D_e'
            threshold = 0.3
            for m in range(number_cell_side):
                #calculate the number of strains in the internal grids
                if 0<m<number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = (distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)+distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-2*distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * micro_decrease_cell
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_increase_bcell = micro_decrease_cell-micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                                    micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                                else:
                                    micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_decrease_cell = micro_decrease_cell
                                    micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                    micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                    micro_increase_fcell = micro_increase_fcell
                                    micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                    micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
                            elif distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the first grid
                elif m == 0:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m+1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_bcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m+1]/(distribute_micro_list_t[ct][i][m+1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_bcell = micro_increase_bcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_bcell = micro_increase_bcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m+1][i] = micro_increase[m+1][i] + micro_increase_bcell
                #calculate the number of strains in the last grid
                elif m == number_cell_side-1:
                    for i in range(number_model):
                        if distribute_micro_list_t[ct][i][m] > 0:
                            if distribute_public_metabolism_list[met][0,m-1] > distribute_public_metabolism_list[met][0,m]:
                                micro_decrease_cell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_decrease_cell = micro_decrease_cell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_decrease_cell = micro_decrease_cell
                                micro_increase_fcell_ratio = min(threshold,(distribute_public_metabolism_list[met][0,m-1]/(distribute_micro_list_t[ct][i][m-1]+0.001)-distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001))/(distribute_public_metabolism_list[met][0,m]/(distribute_micro_list_t[ct][i][m]+0.001)+0.001))
                                micro_increase_fcell = micro_increase_fcell_ratio * distribute_micro_list_t[ct][i][m]
                                micro_increase_fcell = micro_increase_fcell
                                micro_decrease[m][i] = micro_decrease[m][i] + micro_decrease_cell
                                micro_increase[m-1][i] = micro_increase[m-1][i] + micro_increase_fcell
        
            #calculate the number of strains after wandering
            for m in range(number_cell_side):
                for i in range(number_model):
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] - micro_decrease[m][i]
                    distribute_micro_list_t[ct][i][m] = distribute_micro_list_t[ct][i][m] + micro_increase[m][i]
                    distribute_micro_list_t[ct][i][m] = max(0, distribute_micro_list_t[ct][i][m])
                            
                            
        
        #simulate the substrate diffusion by Fick's second law
        for m in range(number_cell_side): 
            if 0<m<number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + ((distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x-(distribute_public_metabolism_list[met][0,m]-distribute_public_metabolism_list[met][0,m-1])/deta_x)/deta_x*D*deta_t
                    if distribute_public_metabolism_list[met][0,m] < 0:
                        print('Warning: the D is too high!')
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == 0:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m+1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
            if m == number_cell_side-1:
                for met in public_metabolism_name_list:
                    distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + (distribute_public_metabolism_list[met][0,m-1]-distribute_public_metabolism_list[met][0,m])/deta_x/deta_x*D*deta_t
                    distribute_public_metabolism_list[met][0,m] = max(0,distribute_public_metabolism_list[met][0,m])
        
        #simulate the metabolism by ETMs
        biomass_value_list = {}
        number_model_range = []
        biomass_strain = {}
        for m in range(number_cell_side):
            biomass_strain[m] = [0, 0]
            if m%2 == 0:
                number_model_range.append([0,1])
            elif m%2 == 1:
                number_model_range.append([1,0])
            
            
            biomass_value_micro = {}
            for i in number_model_range[m]:
                if distribute_micro_list_t[ct][i][m] > 0:
                    public_metabolism_flux_list = {}
                    for j in public_metabolism_name_list:
                        public_metabolism_flux_list.update({j: 0})
                    for j in public_reaction_list[i]:
                        model_list[i]['ub_list'][j] = public_reaction_ub_list[i][j][0,m]
                    model_list[i]['ub_list']['EX_o2_e_reverse'] = min(v_max_o2*o2_concentration/(o2_concentration+k_m_o2), model_list[i]['ub_list']['EX_o2_e_reverse'])   
            
                    biomass_id = biomass_list[i]
                    E_total=parameter_list[i][0]
                    #set the carbon source to glucose
                    substrate_name='EX_glc__D_e_reverse'
                    substrate_value=parameter_list[i][1]
                    biomass_value=growth_list[i]
                    K_value=parameter_list[i][2]

                    #calculate the MDF values of metabolic networks
                    biomass_value_micro[i] = biomass_value
                    obj_name=biomass_list[i]
                    obj_target='maximize'
                    if i == 0:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation0(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        max_biomass_under_mdf=Max_Growth_Rate_Calculation1(model_list[i],obj_name,obj_target,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    biomass_value=max_biomass_under_mdf*0.9
                        
                    #calculate the minimum value of the sum of the fluxes
                    if i == 0:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation0(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
                    elif i == 1:
                        [min_V,Concretemodel]=Min_Flux_Sum_Calculation1(model_list[i],biomass_value,biomass_id,substrate_name,substrate_value,K_value,E_total,0,'gurobi')
            
                    #translating the results of ETMs into a usable form
                    model=model_list[i]['model']
                    reaction_kcat_MW=model_list[i]['reaction_kcat_MW']
                    reaction_g0=model_list[i]['reaction_g0']
                    coef_matrix=model_list[i]['coef_matrix']
                    metabolite_list=model_list[i]['metabolite_list']
                    use_result = Get_Results_Thermodynamics(model,Concretemodel,reaction_kcat_MW,reaction_g0,coef_matrix,metabolite_list)
            
                    if distribute_micro_list_t[ct][i][m] == 0:
                        biomass_strain[m][i] = 0
                    else:
                        biomass_strain[m][i] = use_result['flux'][biomass_list[i]]
                
                    #simulate the fluxes of the public metabolites
                    for rea in public_reaction_list[i]:
                        for met in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(met,rea)]  
                            except:
                                pass
                            else:
                                public_metabolism_flux_list[met] = public_metabolism_flux_list[met] + model_list[i]['coef_matrix'][(met,rea)]* use_result['flux'][rea]
                        
                    #simulate the distribution of the public metabolites
                    distribute_public_metabolism_ori = copy.deepcopy(distribute_public_metabolism_list)
                    distribute_public_metabolism_re = copy.deepcopy(distribute_public_metabolism_ori)
                    for met in public_metabolism_name_list:
                        distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] - public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m]
                        distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                        if distribute_public_metabolism_list[met][0,m] < 0:
                            print(met+'_1: ', distribute_public_metabolism_list[met][0,m])
                                
                    #simulate the multiplication and death rates of strains and the distribution of public metabolites after multiplication or death        
                    death_rate = 0
                    birth_rate = 1
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                bd_rate_met = (distribute_public_metabolism_list[met][0,m] - 0.1)/intracellular_c_dict[met]/distribute_micro_list_t[ct][i][m]
                                if bd_rate_met < 0:
                                    death_rate_lb = (public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] - distribute_public_metabolism_re[met][0,m] + 0.1)/(public_metabolism_flux_list[met] * deta_t * distribute_micro_list_t[ct][i][m] + intracellular_c_dict[met] * distribute_micro_list_t[ct][i][m])
                                    death_rate = max(death_rate, death_rate_lb)
                                else:
                                    birth_rate = min(birth_rate, bd_rate_met)
                    death_rate = min(death_rate, 1)
                    birth_rate = min(birth_rate,1)
                    if death_rate > 0:
                        birth_rate = 0
                        for met in public_metabolism_name_list:
                            distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_list[met][0,m] + public_metabolism_flux_list[met] * deta_t * math.ceil(distribute_micro_list_t[ct][i][m] * death_rate)
                            distribute_public_metabolism_ori[met][0,m] = copy.deepcopy(distribute_public_metabolism_list[met][0,m])
                    
                    for rea in public_reaction_list[i]:
                        if 'reverse' not in rea:
                            met = rea[3::]
                            if met in public_metabolism_name_list:
                            
                                intracellular_c = use_result['met_concentration'][rea].lstrip(';')
                                intracellular_c = intracellular_c.replace(';',',"')
                                intracellular_c = intracellular_c.replace(' :','" :')
                                intracellular_c = '{"' + intracellular_c + '}'
                                intracellular_c_dict = ast.literal_eval(intracellular_c)
                                
                                distribute_public_metabolism_list[met][0,m] = distribute_public_metabolism_ori[met][0,m] - intracellular_c_dict[met]*math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i]) + intracellular_c_dict[met]*math.ceil(death_rate*distribute_micro_list_t[ct][i][m])
                                if distribute_public_metabolism_list[met][0,m] < 0:
                                    print(met+'_2: ', distribute_public_metabolism_list[met][0,m])
                                    distribute_public_metabolism_list[met][0,m] = 0
                        
                    #simulate the distribution of strains after multiplication or death
                    distribute_micro_increase = math.floor(distribute_micro_list_t[ct][i][m]*birth_rate/breed_list[i])
                    distribute_micro_decrease = math.ceil(distribute_micro_list_t[ct][i][m]*death_rate)
                    distribute_micro_list_t[ct][i][m] = max(0,distribute_micro_list_t[ct][i][m] + distribute_micro_increase - distribute_micro_decrease)
            
            #calculate the upperbounds of nutrient uptake rates after substrate diffusion, cell wandering, metabolism, multiplication and death
            biomass_value_list[m] = biomass_value_micro
            
            for i in range(number_model):
                for rea in model_list[i]['reaction_list']:
                    if 'EX_' in rea:
                        for j in public_metabolism_name_list:
                            try:
                                model_list[i]['coef_matrix'][(j,rea)]
                            except:
                                pass
                            else:
                                if model_list[i]['coef_matrix'][(j,rea)] > 0:
                                    public_reaction_ub_list[i][rea][0, m] = v_max[(i,rea)]*distribute_public_metabolism_list[j][0,m]/(distribute_public_metabolism_list[j][0,m]+k_m[(i,rea)])
        print('biomass_strain: ',biomass_strain)           
        
        if ct == 1:
            biomass_strain_initial = copy.deepcopy(biomass_strain)
        
        # calculate the mean number and uniformity of distribution of strains at this iteratio
        for i in range(number_model):
            strain_number = np.mean(distribute_micro_list_t[ct][i])
            strain_various = np.std(distribute_micro_list_t[ct][i])
            number[i].append(strain_number)
            various[i].append(strain_various)
            print('strain_number: ', i, strain_number)
            print('strain_various: ', i, strain_various)
        
        # calculate the slip_r at this iteration
        if ct > 1:
            r = 0
            for i in range(number_model):
                for m in range(number_cell_side):
                    if distribute_micro_list_t[ct-1][i][m] > 0:
                        r = r + abs((distribute_micro_list_t[ct][i][m]-distribute_micro_list_t[ct-1][i][m])/(distribute_micro_list_t[ct-1][i][m])/deta_t)
                    else:
                        r = r + abs(distribute_micro_list_t[ct][i][m]/deta_t)
       
        r_threshold.append(r)
        slip_r = np.mean(r_threshold[-1:])
        fd_r_threshold = r_threshold[1:]
        print('fd_r_threshold: ', fd_r_threshold)
        print('slip_r: ', slip_r)
        
        biomass_x = {}
        biomass_x_initial = {}
        for i in range(number_model):
            biomass_x[i] = [1]*number_cell_side
            if ct == 1:
                biomass_x_initial[i] = np.ones(number_cell_side)
            for m in range(number_cell_side):
                biomass_x[i][m] = distribute_micro_list_t[ct][i][m]*biomass_strain[m][i]
                if ct == 1:
                    biomass_x_initial[i][m] = distribute_micro_list_initial[i][m]*biomass_strain_initial[m][i]
        biomass[ct] = biomass_x
        
        if ct == 1:
            print('_1: ', distribute_micro_list_initial)
            print('_2: ', biomass_strain_initial)
            print('_3: ', biomass_x_initial)
            
    return(biomass_x_initial, biomass, number, various)

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10, 18, 19, 19, 16, 19,  7,  3, 18, 10, 11,  9, 13,  7,  1, 15,  4,
        2,  2,  4,  2, 12, 15, 15, 15, 20, 12,  7, 18, 12]), 1: array([ 3, 10,  8,  7,  4, 16, 12, 15,  9, 13, 16, 18,  9, 18,  6,  8, 12,
        2, 14, 10,  2,  5,  1,  8,  3, 18, 13,  9,  4, 14])}
1
biomass_strain:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0.2857391744693639], 6: [0.6784862515713712, 0.2857391744693639], 7: [0.6784862515713712, 0.2857391744693639], 8: [0.6784862515713712, 0.2857391744693639], 9: [0.6784862515713712, 0.2857391744693639], 10: [0.6784862515713712, 0.2857391744693639], 11: [0.6784862515713712, 0.2857391744693639], 12: [0.6784862515713712, 0.2857391744693639], 13: [0.6784862515713712, 0.2857391744693639], 14: [0.6784862515713712, 0.2857391744693639], 15: [0.67848625157137

biomass_strain:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0.2857391744693639], 6: [0.6784862515713712, 0.2857391744693639], 7: [0.6784862515713712, 0.2857391744693639], 8: [0.6784862515713712, 0.2857391744693639], 9: [0.6784862515713712, 0.2857391744693639], 10: [0.6784862515713712, 0.2857391744693639], 11: [0.6784862515713712, 0.2857391744693639], 12: [0.6784862515713712, 0.2857391744693639], 13: [0.6784862515713712, 0.2857391744693639], 14: [0.6784862515713712, 0.2857391744693639], 15: [0.6784862515713712, 0.2857391744693639], 16: [0.6784862515713712, 0.2857391744693639], 17: [0.6784862515713712, 0.2857391744693639], 18: [0.6784862515713712, 0.2857391744693639], 19: [0.6784862515713712, 0.2857391744693639], 20: [0.6784862515713712, 0.2857391744693639], 21: [0.6784862515713712, 0

biomass_strain:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0.2857391744693639], 6: [0.6784862515713712, 0.2857391744693639], 7: [0.6784862515713712, 0.2857391744693639], 8: [0.6784862515713712, 0.2857391744693639], 9: [0.6784862515713712, 0.2857391744693639], 10: [0.6784862515713712, 0.2857391744693639], 11: [0.6784862515713712, 0.2857391744693639], 12: [0.6784862515713712, 0.2857391744693639], 13: [0.6784862515713712, 0.2857391744693639], 14: [0.6784862515713712, 0.2857391744693639], 15: [0.6784862515713712, 0.2857391744693639], 16: [0.6784862515713712, 0.2857391744693639], 17: [0.6784862515713712, 0.2857391744693639], 18: [0.6784862515713712, 0.2857391744693639], 19: [0.6784862515713712, 0.2857391744693639], 20: [0.6784862515713712, 0], 21: [0.6784862515713712, 0], 22: [0.6784862

glc__D_e_1:  -5.866162087257911
glc__D_e_1:  -24.086512280570485
glc__D_e_1:  -6.958352654098859
glc__D_e_1:  -23.595946852236462
glc__D_e_1:  -7.898671069423267
glc__D_e_1:  -24.96618304504141
glc__D_e_1:  -0.7633626645073658
glc__D_e_1:  -10.556691698339158
glc__D_e_1:  -10.784354930867828
glc__D_e_1:  -0.10574711005503623
glc__D_e_1:  -16.225612901034033
glc__D_e_1:  -13.688250253023767
glc__D_e_1:  -0.4221584340319602
glc__D_e_1:  -22.174375501182197
glc__D_e_1:  -19.751729643386852
glc__D_e_1:  -11.058907697903045
glc__D_e_1:  -28.397780065829963
glc__D_e_1:  -7.079825812843547
glc__D_e_1:  -26.23341298723563
glc__D_e_1:  -9.177810627652555
biomass_strain:  {0: [0.6784862515713712, 0], 1: [0.6784862515713712, 0], 2: [0.6784862515713712, 0], 3: [0.6784862515713712, 0], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0], 6: [0.6784862515713712, 0.2857391744693639], 7: [0.6784862515713712, 0.2857391744693639], 8: [0.6784862515713712, 0.2857391744693639], 9: [0.67

glc__D_e_1:  -1.7693961133199418
glc__D_e_1:  -0.8079881590394112
glc__D_e_1:  -1.462435467347908
glc__D_e_1:  -2.318043821563273
glc__D_e_1:  -1.332120134797845
glc__D_e_1:  -1.6571384910329425
glc__D_e_1:  -0.6025758008065235
glc__D_e_1:  -1.448141823482015
glc__D_e_1:  -1.062462336453692
glc__D_e_1:  -1.613928702022897
glc__D_e_1:  -2.227671509243682
glc__D_e_1:  -5.695230386010513
glc__D_e_1:  -6.563702617535824
glc__D_e_1:  -12.090981211210455
glc__D_e_1:  -5.8771953208818495
glc__D_e_1:  -23.31725440449022
glc__D_e_1:  -15.29094794287843
glc__D_e_1:  -3.885089836522757
glc__D_e_1:  -9.08880004316348
glc__D_e_1:  -1.5001632046852977
glc__D_e_1:  -2.169424489016553
glc__D_e_1:  -2.703766984961378
glc__D_e_1:  -1.1214126008859095
glc__D_e_1:  -0.48982588765556545
glc__D_e_1:  -2.2240099621604275
glc__D_e_1:  -1.994448517243503
biomass_strain:  {0: [0.6784862515713712, 0], 1: [0.6784862515713712, 0], 2: [0.6784862515713712, 0], 3: [0.6784862515713712, 0], 4: [0.6784862515713712, 0], 

glc__D_e_1:  -0.061456851830692716
glc__D_e_1:  -0.3795968451193168
glc__D_e_1:  -1.043534734188545
glc__D_e_1:  -6.2971092507598945
glc__D_e_1:  -11.96176143495209
glc__D_e_1:  -15.673112997937592
glc__D_e_1:  -8.167777868014687
glc__D_e_1:  -9.373941745987425
glc__D_e_1:  -5.970769778248439
glc__D_e_1:  -2.6905238538537537
glc__D_e_1:  -1.374866728042889
glc__D_e_1:  -0.2459188310093643
glc__D_e_1:  -0.13573645157627284
biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0, 0], 6: [0, 0], 7: [0.6784862515713712, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0.6784862515713712, 0], 13: [0.6784862515713712, 0], 14: [0.6784862515713712, 0], 15: [0.6784862515713712, 0], 16: [0.6784862515713712, 0], 17: [0.6784862515713712, 0], 18: [0.6784862515713712, 0], 19: [0.6784862515713712, 0], 20: [0.6784862515713712, 0], 21: [0.6784862515713712, 0], 22: [0.6784862515713712, 0], 23: [0.6784862515713712, 0], 24: [0, 0], 25: [0, 0], 26: [0, 0], 27: [0.67848625157137

biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0, 0], 6: [0, 0], 7: [0, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0, 0], 13: [0, 0], 14: [0, 0], 15: [0, 0], 16: [0, 0], 17: [0, 0], 18: [0, 0], 19: [0, 0], 20: [0, 0], 21: [0, 0], 22: [0, 0], 23: [0, 0], 24: [0, 0], 25: [0, 0], 26: [0, 0], 27: [0, 0], 28: [0, 0], 29: [0, 0]}
strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 690.6764288343236, 368.50436989910685, 411.77663706262774, 444.9439434539711, 449.9976809127489, 399.5441752171249, 432.61741229639165, 447.8490360722646, 404.42185524242456, 377.37309866153697, 393.1630404906224, 398.03811853829484, 283.70324174000234, 316.2024656653215, 202.20495424914776, 189.91299734691276, 195.2750319841559, 206.59892761398189, 160.58534752284754, 212.33907922143214, 137.1857886359165, 93.14285714285714, 62.121212121212125, 90.21428571428572, 46.5, 75.0, 30.0]
slip_r:  30.

In [7]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([19, 11, 18,  9,  2, 10, 13, 15,  5,  1,  9, 15, 15, 15, 15,  8,  2,
        4,  4, 15,  3, 13,  8,  9,  8, 17,  6, 19, 17, 10]), 1: array([14, 13, 16, 10,  8, 16,  2,  3, 15, 16, 19, 11,  3,  6, 10,  3,  2,
        2, 11,  8,  2,  4,  5,  1,  7, 11, 20, 19, 18,  1])}
1
biomass_strain:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0.2857391744693639], 6: [0.6784862515713712, 0.2857391744693639], 7: [0.6784862515713712, 0.2857391744693639], 8: [0.6784862515713712, 0.2857391744693639], 9: [0.6784862515713712, 0.2857391744693639], 10: [0.6784862515713712, 0.2857391744693639], 11: [0.6784862515713712, 0.2857391744693639], 12: [0.6784862515713712, 0.2857391744693639], 13: [0.6784862515713712, 0.2857391744693639], 14: [0.6784862515713712, 0.2857391744693639], 15: [0.67848625157137

biomass_strain:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0.2857391744693639], 6: [0.6784862515713712, 0.2857391744693639], 7: [0.6784862515713712, 0.2857391744693639], 8: [0.6784862515713712, 0.2857391744693639], 9: [0.6784862515713712, 0.2857391744693639], 10: [0.6784862515713712, 0.2857391744693639], 11: [0.6784862515713712, 0.2857391744693639], 12: [0.6784862515713712, 0.2857391744693639], 13: [0.6784862515713712, 0.2857391744693639], 14: [0.6784862515713712, 0.2857391744693639], 15: [0.6784862515713712, 0.2857391744693639], 16: [0.6784862515713712, 0.2857391744693639], 17: [0.6784862515713712, 0.2857391744693639], 18: [0.6784862515713712, 0.2857391744693639], 19: [0.6784862515713712, 0.2857391744693639], 20: [0.6784862515713712, 0.2857391744693639], 21: [0.6784862515713712, 0

biomass_strain:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0.2857391744693639], 2: [0.6784862515713712, 0.2857391744693639], 3: [0.6784862515713712, 0.2857391744693639], 4: [0.6784862515713712, 0.2857391744693639], 5: [0.6784862515713712, 0.2857391744693639], 6: [0.6784862515713712, 0.2857391744693639], 7: [0.6784862515713712, 0.2857391744693639], 8: [0.6784862515713712, 0.2857391744693639], 9: [0.6784862515713712, 0.2857391744693639], 10: [0.6784862515713712, 0.2857391744693639], 11: [0.6784862515713712, 0.2857391744693639], 12: [0.6784862515713712, 0.2857391744693639], 13: [0.6784862515713712, 0.2857391744693639], 14: [0.6784862515713712, 0], 15: [0.6784862515713712, 0], 16: [0.6784862515713712, 0], 17: [0.6784862515713712, 0], 18: [0.6784862515713712, 0], 19: [0.6784862515713712, 0], 20: [0.6784862515713712, 0.2857391744693639], 21: [0.6784862515713712, 0], 22: [0.6784862515713712, 0], 23: [0.6784862515713712, 0], 24: [0.6784862515713712, 0.28573917446936

glc__D_e_1:  -14.256715933020123
glc__D_e_1:  -0.3219500655958054
glc__D_e_1:  -8.968787957943078
glc__D_e_1:  -6.1653584244609805
glc__D_e_1:  -10.926073411984222
glc__D_e_1:  -31.634130660361002
glc__D_e_1:  -1.5687134806509242
glc__D_e_1:  -12.783954293419102
glc__D_e_1:  -31.714016574630406
glc__D_e_1:  -0.31125129805530705
glc__D_e_1:  -23.989466172841514
glc__D_e_1:  -26.464834490364638
glc__D_e_1:  -0.3123774515088513
glc__D_e_1:  -0.1754733664618362
glc__D_e_1:  -22.621875914588042
glc__D_e_1:  -20.66578792875471
glc__D_e_1:  -13.916051399709602
glc__D_e_1:  -10.768552032389005
glc__D_e_1:  -7.918188765302105
glc__D_e_1:  -2.488796490573243
glc__D_e_1:  -23.78312821765495
glc__D_e_1:  -16.57158096936421
glc__D_e_1:  -14.426483505429983
glc__D_e_1:  -5.273093574543458
glc__D_e_1:  -17.426970092337775
glc__D_e_1:  -3.8211531787633324
biomass_strain:  {0: [0.6784862515713712, 0.2857391744693639], 1: [0.6784862515713712, 0], 2: [0.6784862515713712, 0], 3: [0.6784862515713712, 0], 4

glc__D_e_1:  -0.910194456322449
glc__D_e_1:  -0.8007664902754212
glc__D_e_1:  -0.6562944198714198
glc__D_e_1:  -1.3663641808926772
glc__D_e_1:  -1.4590746590659471
glc__D_e_1:  -3.134830745227788
glc__D_e_1:  -1.6099710092008235
glc__D_e_1:  -6.858333700184033
glc__D_e_1:  -3.0297994840113773
glc__D_e_1:  -3.736779856958613
glc__D_e_1:  -6.428428189556046
glc__D_e_1:  -3.037451727038279
glc__D_e_1:  -3.082527673409768
glc__D_e_1:  -8.201378080087055
glc__D_e_1:  -9.152883894824148
glc__D_e_1:  -5.94496281293541
glc__D_e_1:  -13.067841226567174
glc__D_e_1:  -12.09600841728691
glc__D_e_1:  -20.788737916526966
glc__D_e_1:  -15.874196408661643
glc__D_e_1:  -11.993486431860138
glc__D_e_1:  -13.045290599564687
glc__D_e_1:  -12.557241199690202
glc__D_e_1:  -6.306032368665958
glc__D_e_1:  -4.478406105587795
glc__D_e_1:  -1.237245659380116
glc__D_e_1:  -2.0412977198393962
glc__D_e_1:  -0.8238034686748539
glc__D_e_1:  -0.46300823450544837
glc__D_e_1:  -1.1536837553554453
biomass_strain:  {0: [0.

glc__D_e_1:  -0.20004439147289022
glc__D_e_1:  -0.18095975495442124
glc__D_e_1:  -1.0926475604603894
glc__D_e_1:  -0.4023550161302456
glc__D_e_1:  -2.303179970050093
glc__D_e_1:  -1.0424799789586525
glc__D_e_1:  -0.7879678844866367
glc__D_e_1:  -1.3773474162578436
glc__D_e_1:  -0.8760345516699096
glc__D_e_1:  -0.7547268477601223
biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0, 0], 6: [0, 0], 7: [0, 0], 8: [0, 0], 9: [0.6784862515713712, 0], 10: [0, 0], 11: [0.6784862515713712, 0], 12: [0.6784862515713712, 0], 13: [0.6784862515713712, 0], 14: [0.6784862515713712, 0], 15: [0.6784862515713712, 0], 16: [0.6784862515713712, 0], 17: [0.6784862515713712, 0], 18: [0.6784862515713712, 0], 19: [0.6784862515713712, 0], 20: [0.6784862515713712, 0], 21: [0.6784862515713712, 0], 22: [0.6784862515713712, 0], 23: [0.6784862515713712, 0], 24: [0.6784862515713712, 0], 25: [0, 0], 26: [0, 0], 27: [0, 0], 28: [0, 0], 29: [0, 0]}
strain_number:  0 0.7
strain_various:  0 1.069

In [8]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 3, 15, 20, 19,  9, 13, 11,  2,  3, 15,  9, 14,  3,  2, 16, 15, 18,
        9, 18, 14, 13,  2,  4, 13,  3, 13, 13,  1, 10,  1]), 1: array([16,  7,  7, 11, 12,  3,  8,  6, 14, 19,  4, 13,  2,  2, 11, 11, 14,
        8,  6, 12, 19, 20, 19,  7,  7,  9, 20, 18,  1, 16])}
1
biomass_strain:  {0: [0.6459265309330249, 0.2857391744693639], 1: [0.6459265309330249, 0.2857391744693639], 2: [0.6459265309330249, 0.2857391744693639], 3: [0.6459265309330249, 0.2857391744693639], 4: [0.6459265309330249, 0.2857391744693639], 5: [0.6459265309330249, 0.2857391744693639], 6: [0.6459265309330249, 0.2857391744693639], 7: [0.6459265309330249, 0.2857391744693639], 8: [0.6459265309330249, 0.2857391744693639], 9: [0.6459265309330249, 0.2857391744693639], 10: [0.6459265309330249, 0.2857391744693639], 11: [0.6459265309330249, 0.2857391744693639], 12: [0.6459265309330249, 0.2857391744693639], 13: [0.6459265309330249, 0.2857391744693639], 14: [0.6459265309330249, 0.2857391744693639], 15: [0.64592653093302

biomass_strain:  {0: [0.6459265309330249, 0.2857391744693639], 1: [0.6459265309330249, 0.2857391744693639], 2: [0.6459265309330249, 0.2857391744693639], 3: [0.6459265309330249, 0.2857391744693639], 4: [0.6459265309330249, 0.2857391744693639], 5: [0.6459265309330249, 0.2857391744693639], 6: [0.6459265309330249, 0.2857391744693639], 7: [0.6459265309330249, 0.2857391744693639], 8: [0.6459265309330249, 0.2857391744693639], 9: [0.6459265309330249, 0.2857391744693639], 10: [0.6459265309330249, 0.2857391744693639], 11: [0.6459265309330249, 0.2857391744693639], 12: [0.6459265309330249, 0.2857391744693639], 13: [0.6459265309330249, 0.2857391744693639], 14: [0.6459265309330249, 0.2857391744693639], 15: [0.6459265309330249, 0.2857391744693639], 16: [0.6459265309330249, 0.2857391744693639], 17: [0.6459265309330249, 0.2857391744693639], 18: [0.6459265309330249, 0.2857391744693639], 19: [0.6459265309330249, 0.2857391744693639], 20: [0.6459265309330249, 0.2857391744693639], 21: [0.6459265309330249, 0

biomass_strain:  {0: [0.6459265309330249, 0.2857391744693639], 1: [0.6459265309330249, 0.2857391744693639], 2: [0.6459265309330249, 0.2857391744693639], 3: [0.6459265309330249, 0.2857391744693639], 4: [0.6459265309330249, 0], 5: [0.6459265309330249, 0.2857391744693639], 6: [0.6459265309330249, 0.2857391744693639], 7: [0.6459265309330249, 0.2857391744693639], 8: [0.6459265309330249, 0.2857391744693639], 9: [0.6459265309330249, 0.2857391744693639], 10: [0.6459265309330249, 0.2857391744693639], 11: [0.6459265309330249, 0.2857391744693639], 12: [0.6459265309330249, 0.2857391744693639], 13: [0.6459265309330249, 0.2857391744693639], 14: [0.6459265309330249, 0.2857391744693639], 15: [0.6459265309330249, 0.2857391744693639], 16: [0.6459265309330249, 0.2857391744693639], 17: [0.6459265309330249, 0.2857391744693639], 18: [0.6459265309330249, 0.2857391744693639], 19: [0.6459265309330249, 0.2857391744693639], 20: [0.6459265309330249, 0.2857391744693639], 21: [0.6459265309330249, 0.2857391744693639

glc__D_e_1:  -14.091758885564854
glc__D_e_1:  -14.338923807982201
glc__D_e_1:  -10.840013104772936
glc__D_e_1:  -10.699268687179014
glc__D_e_1:  -26.194424090704292
glc__D_e_1:  -25.288392787793807
glc__D_e_1:  -16.681605527176856
glc__D_e_1:  -17.965497475085552
glc__D_e_1:  -5.699763168441496
glc__D_e_1:  -0.23549890482584646
glc__D_e_1:  -6.366668703666068
glc__D_e_1:  -11.64019575362935
glc__D_e_1:  -0.04299317796553925
glc__D_e_1:  -16.20357499246059
glc__D_e_1:  -24.802682141279277
glc__D_e_1:  -29.36001221124959
glc__D_e_1:  -30.913478834566224
glc__D_e_1:  -12.098402116800271
glc__D_e_1:  -12.728038469474305
glc__D_e_1:  -15.605133394495429
glc__D_e_1:  -9.786576187725982
glc__D_e_1:  -0.17863218656317442
glc__D_e_1:  -11.189009915996701
glc__D_e_1:  -17.79870773271898
glc__D_e_1:  -1.405914941872032
glc__D_e_1:  -0.634932923158213
glc__D_e_1:  -14.31148354265194
glc__D_e_1:  -9.545964536393683
glc__D_e_1:  -0.1506705871744236
glc__D_e_1:  -3.5645621659634656
biomass_strain:  {

glc__D_e_1:  -6.69259158877275
glc__D_e_1:  -7.184934755740658
glc__D_e_1:  -28.82116340110399
glc__D_e_1:  -18.552728715119066
glc__D_e_1:  -0.7421409879022886
glc__D_e_1:  -24.016914011612737
glc__D_e_1:  -27.761798063062145
glc__D_e_1:  -1.3050372745634249
glc__D_e_1:  -16.08793019587226
biomass_strain:  {0: [0.6459265309330249, 0], 1: [0.6459265309330249, 0], 2: [0.6459265309330249, 0], 3: [0.6459265309330249, 0], 4: [0.6459265309330249, 0], 5: [0.6459265309330249, 0], 6: [0.6459265309330249, 0], 7: [0.6459265309330249, 0], 8: [0.6459265309330249, 0], 9: [0.6459265309330249, 0], 10: [0.6459265309330249, 0], 11: [0.6459265309330249, 0], 12: [0.6459265309330249, 0], 13: [0.6459265309330249, 0], 14: [0.6459265309330249, 0], 15: [0.6459265309330249, 0], 16: [0.6459265309330249, 0], 17: [0.6459265309330249, 0], 18: [0.6459265309330249, 0], 19: [0.6459265309330249, 0], 20: [0.6459265309330249, 0], 21: [0.6459265309330249, 0], 22: [0.6459265309330249, 0], 23: [0.6459265309330249, 0], 24: 

glc__D_e_1:  -0.08351866605029801
glc__D_e_1:  -1.3830693625165487
glc__D_e_1:  -0.055701177226157306
glc__D_e_1:  -0.34030238332251156
glc__D_e_1:  -0.7900671639981283
glc__D_e_1:  -0.8218879659085678
glc__D_e_1:  -2.1511877428861346
glc__D_e_1:  -5.3343300125718205
glc__D_e_1:  -3.979552155461618
glc__D_e_1:  -4.068506225633615
biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0.6459265309330249, 0], 6: [0.6459265309330249, 0], 7: [0.6459265309330249, 0], 8: [0.6459265309330249, 0], 9: [0.6459265309330249, 0], 10: [0.6459265309330249, 0], 11: [0.6459265309330249, 0], 12: [0.6459265309330249, 0], 13: [0.6459265309330249, 0], 14: [0, 0], 15: [0, 0], 16: [0.6459265309330249, 0], 17: [0, 0], 18: [0, 0], 19: [0.6459265309330249, 0], 20: [0.6459265309330249, 0], 21: [0.6459265309330249, 0], 22: [0.6459265309330249, 0], 23: [0.6459265309330249, 0], 24: [0.6459265309330249, 0], 25: [0.6459265309330249, 0], 26: [0.6459265309330249, 0], 27: [0.6459265309330249, 0], 2

In [6]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([19, 14, 16,  8,  1, 16, 19, 15,  3,  5,  8,  1,  9, 12, 17, 20, 12,
       11, 20, 18, 15, 16, 12,  7, 16, 18, 12,  1,  3,  8]), 1: array([ 6,  5, 20, 13, 14,  6,  4, 17, 20,  3,  2,  6, 11,  7, 18,  1, 19,
       19,  3, 17, 12,  6, 14, 18,  7, 10,  5, 15, 12, 18])}
1
biomass_strain:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0.520733775712074, 0.2857391744693639], 6: [0.520733775712074, 0.2857391744693639], 7: [0.520733775712074, 0.2857391744693639], 8: [0.520733775712074, 0.2857391744693639], 9: [0.520733775712074, 0.2857391744693639], 10: [0.520733775712074, 0.2857391744693639], 11: [0.520733775712074, 0.2857391744693639], 12: [0.520733775712074, 0.2857391744693639], 13: [0.520733775712074, 0.2857391744693639], 14: [0.520733775712074, 0.2857391744693639], 15: [0.520733775712074, 0.2857391744

biomass_strain:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0.520733775712074, 0.2857391744693639], 6: [0.520733775712074, 0.2857391744693639], 7: [0.520733775712074, 0.2857391744693639], 8: [0.520733775712074, 0.2857391744693639], 9: [0.520733775712074, 0.2857391744693639], 10: [0.520733775712074, 0.2857391744693639], 11: [0.520733775712074, 0.2857391744693639], 12: [0.520733775712074, 0.2857391744693639], 13: [0.520733775712074, 0.2857391744693639], 14: [0.520733775712074, 0.2857391744693639], 15: [0.520733775712074, 0.2857391744693639], 16: [0.520733775712074, 0.2857391744693639], 17: [0.520733775712074, 0.2857391744693639], 18: [0.520733775712074, 0.2857391744693639], 19: [0.520733775712074, 0.2857391744693639], 20: [0.520733775712074, 0.2857391744693639], 21: [0.520733775712074, 0.2857391744693639], 22

biomass_strain:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0.520733775712074, 0.2857391744693639], 6: [0.520733775712074, 0.2857391744693639], 7: [0.520733775712074, 0.2857391744693639], 8: [0.520733775712074, 0.2857391744693639], 9: [0.520733775712074, 0.2857391744693639], 10: [0.520733775712074, 0.2857391744693639], 11: [0.520733775712074, 0.2857391744693639], 12: [0.520733775712074, 0.2857391744693639], 13: [0.520733775712074, 0.2857391744693639], 14: [0.520733775712074, 0.2857391744693639], 15: [0.520733775712074, 0.2857391744693639], 16: [0.520733775712074, 0.2857391744693639], 17: [0.520733775712074, 0.2857391744693639], 18: [0.520733775712074, 0.2857391744693639], 19: [0.520733775712074, 0.2857391744693639], 20: [0.520733775712074, 0.2857391744693639], 21: [0.520733775712074, 0.2857391744693639], 22

glc__D_e_1:  -15.952352010158512
glc__D_e_1:  -9.99287494563607
glc__D_e_1:  -16.754004584742244
glc__D_e_1:  -11.601048205579728
glc__D_e_1:  -28.173840516723775
glc__D_e_1:  -1.0423155918235796
glc__D_e_1:  -18.49831463751334
glc__D_e_1:  -16.82999308104909
glc__D_e_1:  -9.923062794667977
glc__D_e_1:  -15.525715490933557
glc__D_e_1:  -12.884587405974312
glc__D_e_1:  -18.008619117269454
glc__D_e_1:  -11.126606071415829
glc__D_e_1:  -7.139740488149895
glc__D_e_1:  -19.276914787692753
glc__D_e_1:  -0.17162848628633753
glc__D_e_1:  -3.8299147675653256
glc__D_e_1:  -8.464826838812847
glc__D_e_1:  -9.776806256838169
glc__D_e_1:  -14.64066673067591
glc__D_e_1:  -0.09860204846309273
glc__D_e_1:  -8.769370522525238
glc__D_e_1:  -17.469737364557353
glc__D_e_1:  -14.889634404955073
glc__D_e_1:  -29.719193709063983
glc__D_e_1:  -0.6350643446383479
glc__D_e_1:  -12.15459739282323
glc__D_e_1:  -13.317746647691692
glc__D_e_1:  -1.6341039354084295
glc__D_e_1:  -5.556951541860158
biomass_strain:  {0:

glc__D_e_1:  -20.410053919924795
glc__D_e_1:  -17.69957142340931
glc__D_e_1:  -16.199215074780334
glc__D_e_1:  -6.795490119885887
glc__D_e_1:  -8.329149726068302
glc__D_e_1:  -2.6752673482366673
glc__D_e_1:  -6.123865118181259
glc__D_e_1:  -3.193891966310327
glc__D_e_1:  -1.5587500816065805
glc__D_e_1:  -0.11455586537540174
glc__D_e_1:  -1.9402893217996984
glc__D_e_1:  -2.520618577290435
glc__D_e_1:  -4.18707388004854
glc__D_e_1:  -2.391158909796528
glc__D_e_1:  -5.219997312120071
glc__D_e_1:  -8.578475126968218
glc__D_e_1:  -3.2566644813313057
glc__D_e_1:  -13.178694140880397
glc__D_e_1:  -3.8790473426918015
glc__D_e_1:  -17.041621590633508
glc__D_e_1:  -0.1260832107241363
glc__D_e_1:  -2.391568605447985
glc__D_e_1:  -22.452937016460403
biomass_strain:  {0: [0.520733775712074, 0], 1: [0.520733775712074, 0], 2: [0.520733775712074, 0], 3: [0.520733775712074, 0], 4: [0.520733775712074, 0], 5: [0.520733775712074, 0], 6: [0.520733775712074, 0], 7: [0.520733775712074, 0], 8: [0.520733775712

glc__D_e_1:  -0.018247525135529186
glc__D_e_1:  -0.08329022108173723
glc__D_e_1:  -0.9248250030987611
glc__D_e_1:  -1.630745720754561
glc__D_e_1:  -1.5608712621694991
glc__D_e_1:  -3.130545413050921
glc__D_e_1:  -0.9088010764641962
glc__D_e_1:  -0.8871024450683129
glc__D_e_1:  -0.2610992148873068
glc__D_e_1:  -0.25071683832252145
glc__D_e_1:  -0.7479801583552057
glc__D_e_1:  -0.6073676150784972
glc__D_e_1:  -1.4206057318476035
glc__D_e_1:  -1.7782976911131159
biomass_strain:  {0: [0.520733775712074, 0], 1: [0, 0], 2: [0, 0], 3: [0.520733775712074, 0], 4: [0.520733775712074, 0], 5: [0.520733775712074, 0], 6: [0.520733775712074, 0], 7: [0.520733775712074, 0], 8: [0.520733775712074, 0], 9: [0.520733775712074, 0], 10: [0.520733775712074, 0], 11: [0.520733775712074, 0], 12: [0.520733775712074, 0], 13: [0.520733775712074, 0], 14: [0, 0], 15: [0.520733775712074, 0], 16: [0, 0], 17: [0.520733775712074, 0], 18: [0, 0], 19: [0, 0], 20: [0, 0], 21: [0, 0], 22: [0.520733775712074, 0], 23: [0.52073

In [7]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([16,  1, 18,  8, 18,  9,  5,  8, 12, 18, 11,  3,  9, 15,  7,  8, 17,
        6,  3,  4,  1,  1,  3, 17, 13,  1, 12, 11, 11,  2]), 1: array([ 7, 11, 15, 13,  2,  8, 12,  9,  3,  2, 15,  2, 19, 10, 10, 20,  5,
       17, 17, 14,  8, 12, 10, 17,  7,  4,  7, 20, 14,  9])}
1
biomass_strain:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0.520733775712074, 0.2857391744693639], 6: [0.520733775712074, 0.2857391744693639], 7: [0.520733775712074, 0.2857391744693639], 8: [0.520733775712074, 0.2857391744693639], 9: [0.520733775712074, 0.2857391744693639], 10: [0.520733775712074, 0.2857391744693639], 11: [0.520733775712074, 0.2857391744693639], 12: [0.520733775712074, 0.2857391744693639], 13: [0.520733775712074, 0.2857391744693639], 14: [0.520733775712074, 0.2857391744693639], 15: [0.520733775712074, 0.2857391744

biomass_strain:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0.520733775712074, 0.2857391744693639], 6: [0.520733775712074, 0.2857391744693639], 7: [0.520733775712074, 0.2857391744693639], 8: [0.520733775712074, 0.2857391744693639], 9: [0.520733775712074, 0.2857391744693639], 10: [0.520733775712074, 0.2857391744693639], 11: [0.520733775712074, 0.2857391744693639], 12: [0.520733775712074, 0.2857391744693639], 13: [0.520733775712074, 0.2857391744693639], 14: [0.520733775712074, 0.2857391744693639], 15: [0.520733775712074, 0.2857391744693639], 16: [0.520733775712074, 0.2857391744693639], 17: [0.520733775712074, 0.2857391744693639], 18: [0.520733775712074, 0.2857391744693639], 19: [0.520733775712074, 0.2857391744693639], 20: [0.520733775712074, 0.2857391744693639], 21: [0.520733775712074, 0.2857391744693639], 22

biomass_strain:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0.2857391744693639], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0.2857391744693639], 4: [0.520733775712074, 0.2857391744693639], 5: [0.520733775712074, 0.2857391744693639], 6: [0.520733775712074, 0.2857391744693639], 7: [0.520733775712074, 0.2857391744693639], 8: [0.520733775712074, 0.2857391744693639], 9: [0.520733775712074, 0.2857391744693639], 10: [0.520733775712074, 0.2857391744693639], 11: [0.520733775712074, 0.2857391744693639], 12: [0.520733775712074, 0.2857391744693639], 13: [0.520733775712074, 0.2857391744693639], 14: [0.520733775712074, 0.2857391744693639], 15: [0.520733775712074, 0.2857391744693639], 16: [0.520733775712074, 0.2857391744693639], 17: [0.520733775712074, 0.2857391744693639], 18: [0.520733775712074, 0.2857391744693639], 19: [0.520733775712074, 0.2857391744693639], 20: [0.520733775712074, 0.2857391744693639], 21: [0.520733775712074, 0.2857391744693639], 22

glc__D_e_1:  -14.793906950032913
glc__D_e_1:  -0.2518422678200949
glc__D_e_1:  -12.642744809165553
glc__D_e_1:  -18.134134082543103
glc__D_e_1:  -0.5499216542012206
glc__D_e_1:  -21.841794102814553
glc__D_e_1:  -32.3361247550567
glc__D_e_1:  -13.595458572588791
glc__D_e_1:  -31.91832089765997
glc__D_e_1:  -30.284914391471297
glc__D_e_1:  -20.025916918156327
glc__D_e_1:  -31.510037348589407
glc__D_e_1:  -20.823312541696296
glc__D_e_1:  -0.865626415657859
glc__D_e_1:  -24.870106602214484
glc__D_e_1:  -14.022564291225667
glc__D_e_1:  -6.950614228011169
biomass_strain:  {0: [0.520733775712074, 0.2857391744693639], 1: [0.520733775712074, 0], 2: [0.520733775712074, 0.2857391744693639], 3: [0.520733775712074, 0], 4: [0.520733775712074, 0], 5: [0.520733775712074, 0], 6: [0.520733775712074, 0], 7: [0.520733775712074, 0], 8: [0.520733775712074, 0], 9: [0.520733775712074, 0.2857391744693639], 10: [0.520733775712074, 0.2857391744693639], 11: [0.520733775712074, 0.2857391744693639], 12: [0.52073377

glc__D_e_1:  -2.189840222263637
glc__D_e_1:  -0.49330748181577133
glc__D_e_1:  -3.8736778216654004
glc__D_e_1:  -1.2587313433105738
glc__D_e_1:  -1.7564290717782214
glc__D_e_1:  -3.1283142418436922
glc__D_e_1:  -4.24289858830708
glc__D_e_1:  -2.6016930179872046
glc__D_e_1:  -4.649839080368374
glc__D_e_1:  -2.7651675981939237
glc__D_e_1:  -5.338746969548458
glc__D_e_1:  -5.018141492925791
glc__D_e_1:  -3.534741861709732
glc__D_e_1:  -2.1360045329008925
glc__D_e_1:  -7.672954951570291
glc__D_e_1:  -4.435483458314582
glc__D_e_1:  -10.810284235425476
glc__D_e_1:  -14.154614047675397
glc__D_e_1:  -0.3999263593531026
glc__D_e_1:  -0.6959606038980723
glc__D_e_1:  -8.690808944129866
glc__D_e_1:  -20.16571344683611
glc__D_e_1:  -10.46979964667232
glc__D_e_1:  -23.981292557309246
glc__D_e_1:  -7.18297864952285
glc__D_e_1:  -10.650239036808939
glc__D_e_1:  -14.751191294719801
biomass_strain:  {0: [0.520733775712074, 0], 1: [0.520733775712074, 0], 2: [0.520733775712074, 0], 3: [0.520733775712074, 

glc__D_e_1:  -1.7295362133812058
glc__D_e_1:  -0.963758466964179
glc__D_e_1:  -4.224507082867936
glc__D_e_1:  -5.6071219314934115
glc__D_e_1:  -9.82762833789665
glc__D_e_1:  -6.49414980339278
glc__D_e_1:  -9.998818971573321
glc__D_e_1:  -6.256762352647623
glc__D_e_1:  -5.220561751023427
glc__D_e_1:  -4.007908506247121
glc__D_e_1:  -0.7652939858785726
glc__D_e_1:  -1.2350758544938605
glc__D_e_1:  -1.0366276977847817
glc__D_e_1:  -0.5546334403916178
glc__D_e_1:  -0.045633046818597256
biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0.520733775712074, 0], 4: [0, 0], 5: [0, 0], 6: [0, 0], 7: [0, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0.520733775712074, 0], 13: [0, 0], 14: [0.520733775712074, 0], 15: [0.520733775712074, 0], 16: [0.520733775712074, 0], 17: [0.520733775712074, 0], 18: [0.520733775712074, 0], 19: [0.520733775712074, 0], 20: [0.520733775712074, 0], 21: [0.520733775712074, 0], 22: [0.520733775712074, 0], 23: [0.520733775712074, 0], 24: [0.520733775712074, 0

biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0, 0], 6: [0, 0], 7: [0, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0, 0], 13: [0, 0], 14: [0, 0], 15: [0, 0], 16: [0, 0], 17: [0, 0], 18: [0, 0], 19: [0, 0], 20: [0, 0], 21: [0, 0], 22: [0, 0], 23: [0, 0], 24: [0, 0], 25: [0, 0], 26: [0, 0], 27: [0, 0], 28: [0, 0], 29: [0, 0]}
strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 848.6213582599031, 374.7415501165501, 356.61447485692855, 427.90356083544935, 378.30304596719077, 431.7669413919413, 404.6291409791411, 457.5832532536287, 445.34835759821965, 410.68339781153236, 356.1308269482721, 382.7822644395029, 427.59935403094295, 426.0074339131171, 319.7293914580142, 242.04556922986268, 222.05079586981148, 179.78663123589885, 232.3613790290907, 200.69337204377769, 232.4318860487339, 202.439552545156, 144.2197802197802, 137.0, 97.75, 102.5, 60.0, 15.0]
slip_r:  15.0
30
bio

In [8]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 2,  1,  7, 18, 17, 15,  4,  7, 13,  4, 11,  7, 14, 15, 15, 20, 20,
        1,  4, 11,  2, 19,  1, 15,  5,  9, 14,  1, 12,  4]), 1: array([20,  1,  4, 17, 10,  2, 17,  8, 20,  8,  7, 15,  1, 19, 16,  8,  6,
        4, 10,  8,  2,  5, 16, 15,  9,  4,  1, 13,  4, 19])}
1
biomass_strain:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.50901728411133

biomass_strain:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.5090172841113372, 0.2857391744693639], 16: [0.5090172841113372, 0.2857391744693639], 17: [0.5090172841113372, 0.2857391744693639], 18: [0.5090172841113372, 0.2857391744693639], 19: [0.5090172841113372, 0.2857391744693639], 20: [0.5090172841113372, 0.2857391744693639], 21: [0.5090172841113372, 0

biomass_strain:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.5090172841113372, 0.2857391744693639], 16: [0.5090172841113372, 0.2857391744693639], 17: [0.5090172841113372, 0.2857391744693639], 18: [0.5090172841113372, 0.2857391744693639], 19: [0.5090172841113372, 0.2857391744693639], 20: [0.5090172841113372, 0.2857391744693639], 21: [0.5090172841113372, 0

glc__D_e_1:  -12.985368694161007
glc__D_e_1:  -12.869806083084505
glc__D_e_1:  -24.48580407439061
glc__D_e_1:  -26.01419086289463
glc__D_e_1:  -11.523208882212307
glc__D_e_1:  -14.0166012798635
glc__D_e_1:  -21.369716837336224
glc__D_e_1:  -15.245798220217536
glc__D_e_1:  -20.519449560232093
glc__D_e_1:  -14.699845217253003
glc__D_e_1:  -22.935479983930676
glc__D_e_1:  -23.5730080456129
glc__D_e_1:  -11.239241020911223
glc__D_e_1:  -2.9030170755375124
glc__D_e_1:  -2.8740522113718043
biomass_strain:  {0: [0.5090172841113372, 0], 1: [0.5090172841113372, 0], 2: [0.5090172841113372, 0], 3: [0.5090172841113372, 0], 4: [0.5090172841113372, 0], 5: [0.5090172841113372, 0], 6: [0.5090172841113372, 0], 7: [0.5090172841113372, 0], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0], 11: [0.5090172841113372, 0], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0], 14: [0.5090172841113372, 0], 15: [0.5090172841

glc__D_e_1:  -27.11927456428091
glc__D_e_1:  -8.729949406666144
glc__D_e_1:  -13.321445791767772
glc__D_e_1:  -8.136949865809854
glc__D_e_1:  -7.532093252652102
glc__D_e_1:  -4.698219488786127
glc__D_e_1:  -3.3211297181248836
glc__D_e_1:  -5.801009091524418
glc__D_e_1:  -4.3537646434821395
glc__D_e_1:  -3.2896745549107793
glc__D_e_1:  -1.9549635036714013
glc__D_e_1:  -0.891117888314937
glc__D_e_1:  -1.878379406720129
glc__D_e_1:  -1.2300727900724135
glc__D_e_1:  -3.022787487052538
glc__D_e_1:  -3.088964528840487
glc__D_e_1:  -4.94319412392432
glc__D_e_1:  -3.465037663464999
glc__D_e_1:  -6.15445222345992
glc__D_e_1:  -5.672236380781857
glc__D_e_1:  -9.8284975934508
glc__D_e_1:  -7.530097392775185
glc__D_e_1:  -8.324181905147022
glc__D_e_1:  -8.423241702023274
glc__D_e_1:  -8.254052498535225
glc__D_e_1:  -30.771846391970865
glc__D_e_1:  -22.065950554507204
glc__D_e_1:  -13.239553072766494
glc__D_e_1:  -36.35898241063899
glc__D_e_1:  -21.834022534586158
biomass_strain:  {0: [0.5090172841

glc__D_e_1:  -1.4403220666799696
glc__D_e_1:  -0.6426175533286866
glc__D_e_1:  -1.571629738880341
glc__D_e_1:  -0.8705214127991633
glc__D_e_1:  -0.19269719642019079
glc__D_e_1:  -0.3534600778886979
glc__D_e_1:  -0.4533953965592439
glc__D_e_1:  -0.29936523639276935
glc__D_e_1:  -0.46631040355232156
glc__D_e_1:  -0.6384568040437384
glc__D_e_1:  -1.1675028080859247
glc__D_e_1:  -1.3074143812132908
glc__D_e_1:  -0.9058995121797335
glc__D_e_1:  -3.114695351250774
glc__D_e_1:  -3.076907902595452
glc__D_e_1:  -4.585427522025333
biomass_strain:  {0: [0.5090172841113372, 0], 1: [0.5090172841113372, 0], 2: [0.5090172841113372, 0], 3: [0.5090172841113372, 0], 4: [0.5090172841113372, 0], 5: [0.5090172841113372, 0], 6: [0.5090172841113372, 0], 7: [0.5090172841113372, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0, 0], 13: [0, 0], 14: [0.5090172841113372, 0], 15: [0, 0], 16: [0, 0], 17: [0, 0], 18: [0.5090172841113372, 0], 19: [0.5090172841113372, 0], 20: [0.5090172841113372, 0], 21: [0.50

In [9]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 9, 12, 12,  8,  7, 20, 16,  3,  4,  9, 11,  8, 18,  1, 18,  9, 20,
        8,  9,  6, 18,  3, 11,  5, 10, 12, 16, 14,  4, 12]), 1: array([15,  7, 10,  7,  1,  4, 12,  9,  9,  7, 18, 12,  8,  1,  9, 10, 12,
        5,  1,  3,  8,  5, 18, 16,  3, 15,  7, 18, 12,  2])}
1
biomass_strain:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.50901728411133

biomass_strain:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.5090172841113372, 0.2857391744693639], 16: [0.5090172841113372, 0.2857391744693639], 17: [0.5090172841113372, 0.2857391744693639], 18: [0.5090172841113372, 0.2857391744693639], 19: [0.5090172841113372, 0.2857391744693639], 20: [0.5090172841113372, 0.2857391744693639], 21: [0.5090172841113372, 0

biomass_strain:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.5090172841113372, 0.2857391744693639], 16: [0.5090172841113372, 0.2857391744693639], 17: [0.5090172841113372, 0.2857391744693639], 18: [0.5090172841113372, 0.2857391744693639], 19: [0.5090172841113372, 0.2857391744693639], 20: [0.5090172841113372, 0.2857391744693639], 21: [0.5090172841113372, 0

glc__D_e_1:  -27.903223480270366
glc__D_e_1:  -22.172237974255957
glc__D_e_1:  -21.26781313103061
glc__D_e_1:  -28.475382219339384
glc__D_e_1:  -19.425992075203347
glc__D_e_1:  -26.519402043942407
glc__D_e_1:  -5.376099515773706
glc__D_e_1:  -28.63700402456164
glc__D_e_1:  -8.466046452714956
glc__D_e_1:  -32.09784599202554
glc__D_e_1:  -17.768817992710254
glc__D_e_1:  -24.382267209691122
glc__D_e_1:  -35.36342938151301
glc__D_e_1:  -18.892548008404738
glc__D_e_1:  -17.40736495654453
glc__D_e_1:  -25.288985517217036
glc__D_e_1:  -36.1625602115125
glc__D_e_1:  -25.758172793231815
glc__D_e_1:  -20.000447743572185
glc__D_e_1:  -32.53279633645437
glc__D_e_1:  -13.196971214776013
glc__D_e_1:  -24.85375199331697
glc__D_e_1:  -21.36330328269836
glc__D_e_1:  -16.728613069817463
glc__D_e_1:  -24.43023232734101
glc__D_e_1:  -19.334218026481768
glc__D_e_1:  -14.93571144530324
glc__D_e_1:  -0.4253508348694801
glc__D_e_1:  -28.546420495503728
glc__D_e_1:  -33.90906927723817
glc__D_e_1:  -16.32704571

glc__D_e_1:  -9.70116147449787
glc__D_e_1:  -5.15576495175343
glc__D_e_1:  -7.319434837365446
biomass_strain:  {0: [0.5090172841113372, 0], 1: [0.5090172841113372, 0], 2: [0.5090172841113372, 0], 3: [0.5090172841113372, 0], 4: [0.5090172841113372, 0], 5: [0.5090172841113372, 0], 6: [0.5090172841113372, 0], 7: [0.5090172841113372, 0], 8: [0.5090172841113372, 0], 9: [0.5090172841113372, 0], 10: [0.5090172841113372, 0], 11: [0.5090172841113372, 0], 12: [0.5090172841113372, 0], 13: [0.5090172841113372, 0], 14: [0.5090172841113372, 0], 15: [0.5090172841113372, 0], 16: [0.5090172841113372, 0], 17: [0.5090172841113372, 0], 18: [0.5090172841113372, 0], 19: [0.5090172841113372, 0], 20: [0.5090172841113372, 0], 21: [0.5090172841113372, 0], 22: [0.5090172841113372, 0], 23: [0.5090172841113372, 0], 24: [0.5090172841113372, 0], 25: [0.5090172841113372, 0], 26: [0.5090172841113372, 0], 27: [0.5090172841113372, 0], 28: [0.5090172841113372, 0], 29: [0.5090172841113372, 0]}
strain_number:  0 11.4
strai

glc__D_e_1:  -0.7451458289891368
glc__D_e_1:  -0.24640446870356836
glc__D_e_1:  -0.12819893055625864
glc__D_e_1:  -0.4765344744759361
glc__D_e_1:  -0.2771702321703393
glc__D_e_1:  -0.3412628744479629
glc__D_e_1:  -0.7500517465926514
glc__D_e_1:  -0.347617086784508
glc__D_e_1:  -0.21155110733651128
glc__D_e_1:  -0.5768053253238186
glc__D_e_1:  -0.6112017178192033
glc__D_e_1:  -1.3104396935034224
glc__D_e_1:  -0.728520398260593
glc__D_e_1:  -0.2630618081672745
glc__D_e_1:  -0.11128599890689972
glc__D_e_1:  -0.3547558988289091
biomass_strain:  {0: [0.5090172841113372, 0], 1: [0.5090172841113372, 0], 2: [0.5090172841113372, 0], 3: [0.5090172841113372, 0], 4: [0.5090172841113372, 0], 5: [0.5090172841113372, 0], 6: [0.5090172841113372, 0], 7: [0.5090172841113372, 0], 8: [0.5090172841113372, 0], 9: [0.5090172841113372, 0], 10: [0.5090172841113372, 0], 11: [0.5090172841113372, 0], 12: [0.5090172841113372, 0], 13: [0.5090172841113372, 0], 14: [0.5090172841113372, 0], 15: [0, 0], 16: [0.50901728

In [10]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 8, 17, 12, 16,  3,  6,  7,  4,  3,  8, 17,  7,  7, 16,  3, 11, 14,
       10, 17,  4,  5,  7, 20,  4, 12,  9, 12, 19, 11, 18]), 1: array([ 2,  1,  6, 15,  2, 10, 20, 13, 16, 13, 18,  3, 11,  2,  3, 20,  9,
       11,  3, 12,  8, 10, 18, 19,  5, 15, 17, 13, 16,  9])}
1
biomass_strain:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.50901728411133

biomass_strain:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.5090172841113372, 0.2857391744693639], 16: [0.5090172841113372, 0.2857391744693639], 17: [0.5090172841113372, 0.2857391744693639], 18: [0.5090172841113372, 0.2857391744693639], 19: [0.5090172841113372, 0.2857391744693639], 20: [0.5090172841113372, 0.2857391744693639], 21: [0.5090172841113372, 0

biomass_strain:  {0: [0.5090172841113372, 0.2857391744693639], 1: [0.5090172841113372, 0.2857391744693639], 2: [0.5090172841113372, 0.2857391744693639], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.5090172841113372, 0.2857391744693639], 16: [0.5090172841113372, 0.2857391744693639], 17: [0.5090172841113372, 0.2857391744693639], 18: [0.5090172841113372, 0.2857391744693639], 19: [0.5090172841113372, 0.2857391744693639], 20: [0.5090172841113372, 0.2857391744693639], 21: [0.5090172841113372, 0

glc__D_e_1:  -0.8754741762429867
glc__D_e_1:  -15.097236139496747
glc__D_e_1:  -10.310888721687961
glc__D_e_1:  -1.0229125573465936
glc__D_e_1:  -1.1930060696062377
glc__D_e_1:  -4.538795249240833
biomass_strain:  {0: [0.5090172841113372, 0], 1: [0.5090172841113372, 0], 2: [0.5090172841113372, 0], 3: [0.5090172841113372, 0.2857391744693639], 4: [0.5090172841113372, 0.2857391744693639], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0.2857391744693639], 13: [0.5090172841113372, 0.2857391744693639], 14: [0.5090172841113372, 0.2857391744693639], 15: [0.5090172841113372, 0.2857391744693639], 16: [0.5090172841113372, 0.2857391744693639], 17: [0.5090172841113372, 0.2857391744693639], 18: [0.50901728411

glc__D_e_1:  -7.532217163555649
glc__D_e_1:  -16.98514031759242
glc__D_e_1:  -4.490000465601348
glc__D_e_1:  -16.90325904516361
glc__D_e_1:  -3.834211541859188
glc__D_e_1:  -8.165553083012306
glc__D_e_1:  -4.887988400994607
glc__D_e_1:  -7.119956356820472
glc__D_e_1:  -0.9968962294911532
glc__D_e_1:  -3.2248089066632275
glc__D_e_1:  -3.982359327435595
glc__D_e_1:  -3.369808839365138
glc__D_e_1:  -2.26216155591333
biomass_strain:  {0: [0.5090172841113372, 0], 1: [0.5090172841113372, 0], 2: [0.5090172841113372, 0], 3: [0.5090172841113372, 0], 4: [0.5090172841113372, 0], 5: [0.5090172841113372, 0.2857391744693639], 6: [0.5090172841113372, 0.2857391744693639], 7: [0.5090172841113372, 0.2857391744693639], 8: [0.5090172841113372, 0.2857391744693639], 9: [0.5090172841113372, 0.2857391744693639], 10: [0.5090172841113372, 0.2857391744693639], 11: [0.5090172841113372, 0.2857391744693639], 12: [0.5090172841113372, 0], 13: [0.5090172841113372, 0], 14: [0.5090172841113372, 0], 15: [0.50901728411133

glc__D_e_1:  -1.2598293026009642
glc__D_e_1:  -1.8357626203198278
glc__D_e_1:  -0.915960958478734
glc__D_e_1:  -2.314604085910716
glc__D_e_1:  -3.3112920217085975
glc__D_e_1:  -3.786677801272729
glc__D_e_1:  -1.370428674864285
glc__D_e_1:  -6.361140426818267
glc__D_e_1:  -1.7750959259501302
glc__D_e_1:  -1.4270198139096046
glc__D_e_1:  -3.6641846246794083
glc__D_e_1:  -3.061788671454009
glc__D_e_1:  -0.8359197845910384
glc__D_e_1:  -1.4524672527677356
glc__D_e_1:  -1.584199963519247
glc__D_e_1:  -1.388657645682353
glc__D_e_1:  -1.4068457323400791
glc__D_e_1:  -0.9104742375467179
glc__D_e_1:  -0.8014812301622674
glc__D_e_1:  -0.9910413928241586
glc__D_e_1:  -0.7251442629323182
glc__D_e_1:  -1.2057763556408474
glc__D_e_1:  -0.05919228444139418
biomass_strain:  {0: [0.5090172841113372, 0], 1: [0.5090172841113372, 0], 2: [0.5090172841113372, 0], 3: [0.5090172841113372, 0], 4: [0.5090172841113372, 0], 5: [0.5090172841113372, 0], 6: [0.5090172841113372, 0], 7: [0.5090172841113372, 0], 8: [0.

In [6]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.02, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([16, 16,  1,  4,  6, 13,  5, 12, 19,  5, 19, 20, 13, 13,  1,  4,  5,
       11,  4, 12, 20,  3,  1,  1, 11,  7, 12,  9, 17,  7]), 1: array([ 1, 17,  2,  2, 19, 15, 12, 16,  9,  4,  1, 12,  8, 15,  3,  2,  7,
       10, 15,  7,  2,  9, 14,  8,  9, 14,  8,  6, 12, 12])}
1
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [0.49885752915679404, 0.28573917446

glc__D_e_1:  -6.563294186773454
glc__D_e_1:  -13.183187716666785
glc__D_e_1:  -7.349914966059245
glc__D_e_1:  -25.49290424879842
glc__D_e_1:  -19.339089421522953
glc__D_e_1:  -19.143205193359737
glc__D_e_1:  -28.15693140846758
glc__D_e_1:  -15.407627617648014
glc__D_e_1:  -25.107347950161635
glc__D_e_1:  -15.28319150951953
glc__D_e_1:  -12.557376838352692
glc__D_e_1:  -22.518725802587166
glc__D_e_1:  -12.059572185268243
glc__D_e_1:  -10.205585077277995
glc__D_e_1:  -6.9817823818202385
glc__D_e_1:  -15.838864724203331
glc__D_e_1:  -0.10615514375923252
glc__D_e_1:  -11.809253918655632
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885

glc__D_e_1:  -7.191181991211753
glc__D_e_1:  -4.688953592645724
glc__D_e_1:  -9.142144387338996
glc__D_e_1:  -3.915424815478604
glc__D_e_1:  -5.519431793466903
glc__D_e_1:  -6.0859567462301305
glc__D_e_1:  -3.4092120772400887
glc__D_e_1:  -4.347665421280228
glc__D_e_1:  -2.510244835463544
glc__D_e_1:  -1.9787428621106131
glc__D_e_1:  -0.5638623551731722
glc__D_e_1:  -2.405589948618092
glc__D_e_1:  -2.5892153053784153
glc__D_e_1:  -4.356795130348045
glc__D_e_1:  -13.668157362772977
glc__D_e_1:  -7.532068546135818
glc__D_e_1:  -27.65992386677779
glc__D_e_1:  -23.149757390959085
glc__D_e_1:  -35.951427548485334
glc__D_e_1:  -33.02704015280865
glc__D_e_1:  -26.55693876435828
glc__D_e_1:  -25.648042565805955
glc__D_e_1:  -35.165499463968565
glc__D_e_1:  -17.05605203718828
glc__D_e_1:  -17.374844276237297
glc__D_e_1:  -7.23314663649873
glc__D_e_1:  -5.504153456957539
glc__D_e_1:  -2.5935537781523834
glc__D_e_1:  -3.0085772768401493
glc__D_e_1:  -1.2164641907779847
biomass_strain:  {0: [0.498

glc__D_e_1:  -0.20600019427635885
glc__D_e_1:  -0.5768816219525832
glc__D_e_1:  -1.3956515094621107
glc__D_e_1:  -2.180385351408142
glc__D_e_1:  -3.669793413835712
glc__D_e_1:  -4.974814625745193
glc__D_e_1:  -3.1217279332418064
glc__D_e_1:  -2.839104943239003
glc__D_e_1:  -4.542225992423239
glc__D_e_1:  -5.7364412074061155
glc__D_e_1:  -3.5043848151489043
glc__D_e_1:  -1.9093930735006097
glc__D_e_1:  -1.0962492016971233
glc__D_e_1:  -0.9178739111523772
glc__D_e_1:  -0.15157382958926147
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0, 0], 13: [0, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: [0.49885752915679404, 0], 17: [0.49885752915679404, 0], 18: [0.49885752915679404, 0], 19: [0.49885752915679404, 0], 20: [0.49885752915679404, 

biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0, 0], 6: [0, 0], 7: [0, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0, 0], 13: [0, 0], 14: [0, 0], 15: [0, 0], 16: [0, 0], 17: [0, 0], 18: [0, 0], 19: [0, 0], 20: [0, 0], 21: [0, 0], 22: [0, 0], 23: [0, 0], 24: [0, 0], 25: [0, 0], 26: [0, 0], 27: [0, 0], 28: [0, 0], 29: [0, 0]}
strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 740.086694587662, 410.3683525839486, 392.4130036630036, 439.59933122433125, 448.7439230619409, 424.1435319066899, 465.12040796379034, 503.57734814391165, 506.0738124809897, 374.5035778249456, 378.88346843835564, 360.82256128364844, 315.4005832188101, 230.03118811264815, 189.80715668808216, 198.75744963553777, 167.80592357335954, 179.37811950976777, 197.68446618275743, 198.81253635628636, 252.9238297905031, 169.0264514896868, 156.28571428571428, 107.5, 57.5, 82.5, 15.0, 15.0]
slip_r:  15.0
30
bi

In [7]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29*3)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 9,  4,  8,  4,  9, 16, 12, 19,  2, 12, 15, 12, 12,  1, 19, 10,  9,
        4,  6, 20,  1,  9,  2,  6, 19, 14,  9,  7, 11,  9]), 1: array([19, 13, 17, 15, 15,  2, 10,  8, 19, 15,  9, 16, 16, 17, 10, 11,  6,
        8,  5,  8,  3,  5, 18,  2,  3,  4, 11,  4, 15, 12])}
1
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

glc__D_e_1:  -7.823811252217741
glc__D_e_1:  -5.388471463759133
glc__D_e_1:  -19.1307701311761
glc__D_e_1:  -23.968434494027704
glc__D_e_1:  -1.6489092452353304
glc__D_e_1:  -1.8697551571806956
glc__D_e_1:  -15.654240780599569
glc__D_e_1:  -27.991624005908662
glc__D_e_1:  -2.448406717589453
glc__D_e_1:  -1.3058950251237214
glc__D_e_1:  -12.4886470878895
glc__D_e_1:  -14.273002441469593
glc__D_e_1:  -0.406883417028423
glc__D_e_1:  -0.4241276538907399
glc__D_e_1:  -20.973120535007663
glc__D_e_1:  -5.8172110133915345
glc__D_e_1:  -0.40449821330156865
glc__D_e_1:  -14.741331205388185
glc__D_e_1:  -12.223240729972932
glc__D_e_1:  -0.25723483304092704
glc__D_e_1:  -2.594092789586181
glc__D_e_1:  -14.952428435476506
glc__D_e_1:  -0.41763536260282486
glc__D_e_1:  -24.623827642469784
glc__D_e_1:  -4.4004014595076235
glc__D_e_1:  -5.73998918024806
glc__D_e_1:  -8.881691144051107
glc__D_e_1:  -6.265517648996475
glc__D_e_1:  -22.195365511212493
glc__D_e_1:  -16.020385649496976
glc__D_e_1:  -15.205

glc__D_e_1:  -6.825371810089605
glc__D_e_1:  -1.50126972876552
glc__D_e_1:  -7.5101877909837444
glc__D_e_1:  -1.258156799578957
glc__D_e_1:  -5.003540528232911
glc__D_e_1:  -5.323621029649825
glc__D_e_1:  -3.6881944275580194
glc__D_e_1:  -1.9584203774758033
glc__D_e_1:  -4.756319894645846
glc__D_e_1:  -2.417843273037923
glc__D_e_1:  -5.507026917003113
glc__D_e_1:  -5.335960444723968
glc__D_e_1:  -7.7916912690389735
glc__D_e_1:  -11.856519854505887
glc__D_e_1:  -23.442310606915914
glc__D_e_1:  -8.04575099594777
glc__D_e_1:  -24.910525034981568
glc__D_e_1:  -23.613338532845706
glc__D_e_1:  -23.874179236955474
glc__D_e_1:  -19.20218259922731
glc__D_e_1:  -7.884780847189268
glc__D_e_1:  -14.099274668356472
glc__D_e_1:  -9.323819066582537
glc__D_e_1:  -10.552953409339588
glc__D_e_1:  -7.0406038706631735
glc__D_e_1:  -7.187136869098689
glc__D_e_1:  -5.185043048250007
glc__D_e_1:  -9.798130796057986
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.285

glc__D_e_1:  -1.4443709931075341
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: [0.49885752915679404, 0], 17: [0.49885752915679404, 0], 18: [0.49885752915679404, 0], 19: [0.49885752915679404, 0], 20: [0.49885752915679404, 0], 21: [0.49885752915679404, 0], 22: [0.49885752915679404, 0], 23: [0.49885752915679404, 0], 24: [0.49885752915679404, 0], 25: [0.49885752915679404, 0], 26: [0.49885752915679404, 0], 27: [0.49885752915679404, 0], 28: [0.49885752915679404, 0], 29: [0.49885752915679404, 0]}
strain_number:  0 3.033333333333333
strain_various:  0 2.442

biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0, 0], 6: [0, 0], 7: [0, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0, 0], 13: [0, 0], 14: [0, 0], 15: [0, 0], 16: [0, 0], 17: [0, 0], 18: [0, 0], 19: [0, 0], 20: [0, 0], 21: [0, 0], 22: [0, 0], 23: [0, 0], 24: [0, 0], 25: [0, 0], 26: [0, 0], 27: [0, 0], 28: [0, 0], 29: [0, 0]}
strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 896.9357796486279, 387.1215144504619, 373.43507840199027, 434.1414573558537, 482.6324557021925, 414.73990125371705, 436.26449881706674, 477.71160251746, 446.94680212844355, 337.5154007332861, 403.8126024714842, 388.95775608581533, 378.4405260208023, 354.97978148245465, 293.0860372620602, 286.1486317271019, 173.68713359306224, 235.7111209868563, 191.64898259635103, 256.21853146853147, 209.35714285714286, 207.75, 143.75, 90.0, 45.0, 0.0]
slip_r:  0.0
The initial distribution of microorganism is: 

In [8]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29*5)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([14, 12,  7, 11, 12,  9, 19, 14,  1,  5, 10,  8,  3, 17, 16,  5, 10,
       14, 18,  1,  1, 13, 19, 12, 17, 11,  7, 13, 17, 15]), 1: array([ 6,  7,  1,  3, 16,  5,  4, 13,  2,  9, 10,  6, 20, 10,  3, 12, 19,
       16, 15, 20, 12, 20,  8,  4, 18,  6, 11,  2, 10, 19])}
1
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [0.49885752915679404, 0.28573917446

glc__D_e_1:  -10.745265345180862
glc__D_e_1:  -21.991952183429966
glc__D_e_1:  -0.5240995983357006
glc__D_e_1:  -9.188168725147948
glc__D_e_1:  -26.773454451099756
glc__D_e_1:  -0.7445617296136327
glc__D_e_1:  -14.159128077106331
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 

glc__D_e_1:  -4.595882311061637
glc__D_e_1:  -5.913986170593527
glc__D_e_1:  -3.332431387628406
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: [0.49885752915679404, 0], 17: [0.49885752915679404, 0], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0], 20: [0.49885752915679404, 0.2857391744693639], 21: [0.49885752915679404, 0], 22: [0.49885752915679404, 0], 23: [0.49885752915679404, 0], 24: [0.49885752915679404, 0], 25: [0.49885752915679404, 0], 26: [0.49885752915679404, 0], 27: [0.49885752915679404, 0], 28: [0.498857529156794

glc__D_e_1:  -1.0937624788680536
glc__D_e_1:  -0.230502820111034
glc__D_e_1:  -0.7164481645023788
glc__D_e_1:  -1.0661718206287683
glc__D_e_1:  -0.6645678399389734
glc__D_e_1:  -1.5513731613641244
glc__D_e_1:  -1.4301932425292636
glc__D_e_1:  -2.0408815106265648
glc__D_e_1:  -1.5902583154814174
glc__D_e_1:  -5.395242729280205
glc__D_e_1:  -4.477545505834729
glc__D_e_1:  -2.9527992576609976
glc__D_e_1:  -2.516590693834442
glc__D_e_1:  -1.0068401805081788
glc__D_e_1:  -0.9110755309930054
glc__D_e_1:  -0.16134638674255863
glc__D_e_1:  -0.6600129595136788
glc__D_e_1:  -0.17714172356434643
glc__D_e_1:  -0.6191669073980947
glc__D_e_1:  -0.38310645211135474
glc__D_e_1:  -0.29905009075594147
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0],

In [9]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([10,  9,  7, 17,  8, 16, 12, 17,  8, 10,  6, 14,  3, 16,  4,  2, 16,
        1, 16, 13, 17, 19, 17,  8,  8,  2,  8, 15,  4,  9]), 1: array([18,  8,  6,  8,  4,  2, 16, 10,  5,  9, 19, 15,  3,  3, 11, 18,  6,
       14,  2,  2,  1, 11, 20,  9, 13, 16, 13, 13,  4,  9])}
1
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

glc__D_e_1:  -35.036803941371815
glc__D_e_1:  -0.5338022600594079
glc__D_e_1:  -25.245568838221285
glc__D_e_1:  -19.70999913872888
glc__D_e_1:  -22.802604457219534
glc__D_e_1:  -28.131984815512055
glc__D_e_1:  -20.04478835243571
glc__D_e_1:  -25.693621140074313
glc__D_e_1:  -17.505805945712023
glc__D_e_1:  -15.02507358057909
glc__D_e_1:  -16.138341832626363
glc__D_e_1:  -12.6778242620246
glc__D_e_1:  -15.346140054128217
glc__D_e_1:  -2.4800663135891696
glc__D_e_1:  -17.765959366809962
glc__D_e_1:  -7.993075896469389
glc__D_e_1:  -32.07430268724444
glc__D_e_1:  -20.891126878347553
glc__D_e_1:  -26.276219553556537
glc__D_e_1:  -9.64559842590279
glc__D_e_1:  -25.82844546591419
glc__D_e_1:  -20.112163662800047
glc__D_e_1:  -14.481408715584207
glc__D_e_1:  -16.302665432796665
glc__D_e_1:  -16.87170150873115
glc__D_e_1:  -22.128433819119405
glc__D_e_1:  -30.253898652966434
glc__D_e_1:  -9.069929080655957
glc__D_e_1:  -0.4335242410391551
glc__D_e_1:  -7.425574536345778
glc__D_e_1:  -0.9228110

glc__D_e_1:  -9.639402333701295
glc__D_e_1:  -22.37758833361235
glc__D_e_1:  -5.861401925394862
glc__D_e_1:  -12.144949194081262
glc__D_e_1:  -4.85479646972834
glc__D_e_1:  -15.180353507504288
glc__D_e_1:  -7.133104899317279
glc__D_e_1:  -7.409296110753462
glc__D_e_1:  -4.293922267728062
glc__D_e_1:  -3.3348744042820924
glc__D_e_1:  -5.122956407810366
glc__D_e_1:  -3.4129353215984493
glc__D_e_1:  -2.4526263681839615
glc__D_e_1:  -5.405753440926764
glc__D_e_1:  -4.195664067884637
glc__D_e_1:  -8.162565994288814
glc__D_e_1:  -11.33096734385484
glc__D_e_1:  -8.00855102961274
glc__D_e_1:  -18.502159867055468
glc__D_e_1:  -7.550720329170864
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915

glc__D_e_1:  -0.7795542380461702
glc__D_e_1:  -0.1804527581610284
glc__D_e_1:  -0.237926475762742
glc__D_e_1:  -0.2952653912296566
glc__D_e_1:  -0.18378926400234175
glc__D_e_1:  -0.3403403837926141
glc__D_e_1:  -0.3121852826579674
glc__D_e_1:  -0.3564899596064582
glc__D_e_1:  -0.052872372362006925
glc__D_e_1:  -0.4401799777005615
glc__D_e_1:  -1.0388328583038797
glc__D_e_1:  -0.8713016138717711
glc__D_e_1:  -1.0099012727874206
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: [0.49885752915679404, 0], 17: [0.49885752915679404, 0], 18: [0.49885752915679404, 0], 19: [0.49

In [10]:
#35
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.29)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7,  9, 14, 10, 19,  2, 16, 17, 16, 16,  6,  2, 12, 11, 12, 18,  3,
       13,  8, 10,  8,  4,  3, 19, 16, 14, 14,  1, 10, 17]), 1: array([ 6, 14, 15,  1, 19, 13, 18, 14,  1,  4, 13, 15, 15, 15,  3,  1, 18,
        1, 15, 18,  6,  9, 18, 20,  8,  4,  7,  9, 17, 16])}
1
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

glc__D_e_1:  -0.2068802446812097
glc__D_e_1:  -10.348938883185753
glc__D_e_1:  -17.325870569962902
glc__D_e_1:  -0.4190581212604947
glc__D_e_1:  -0.059447677712108904
glc__D_e_1:  -15.404973437522841
glc__D_e_1:  -26.20385550303218
glc__D_e_1:  -0.17496278154605704
glc__D_e_1:  -19.465434671396977
glc__D_e_1:  -24.209977223284888
glc__D_e_1:  -0.5531038776274926
glc__D_e_1:  -10.098848914331342
glc__D_e_1:  -21.17599147206935
glc__D_e_1:  -1.8971596475382135
glc__D_e_1:  -3.921096443683062
glc__D_e_1:  -2.0347525010212477
glc__D_e_1:  -9.049891146133579
glc__D_e_1:  -14.792409790165065
glc__D_e_1:  -0.9262907657238948
glc__D_e_1:  -16.987608961213198
glc__D_e_1:  -19.14392351637793
glc__D_e_1:  -0.7167643555449033
glc__D_e_1:  -0.2447037934396601
glc__D_e_1:  -7.264682159160488
glc__D_e_1:  -36.18405438848911
glc__D_e_1:  -2.3704294910842947
glc__D_e_1:  -0.5950933100678737
glc__D_e_1:  -17.501659206270464
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.4988575291

glc__D_e_1:  -7.105180750982197
glc__D_e_1:  -0.8407952871941227
glc__D_e_1:  -1.2775110289066265
glc__D_e_1:  -8.565765468712208
glc__D_e_1:  -0.11235924436100397
glc__D_e_1:  -3.979421813418055
glc__D_e_1:  -5.4639128206598855
glc__D_e_1:  -0.05120002056991968
glc__D_e_1:  -3.098187713630571
glc__D_e_1:  -7.474066621451301
glc__D_e_1:  -5.8628931188506925
glc__D_e_1:  -5.099622562023461
glc__D_e_1:  -7.114185497243628
glc__D_e_1:  -6.79829047382696
glc__D_e_1:  -9.911830275423466
glc__D_e_1:  -13.140130246102107
glc__D_e_1:  -6.885756939480957
glc__D_e_1:  -9.467299136281108
glc__D_e_1:  -12.835008430172577
glc__D_e_1:  -17.81411905755594
glc__D_e_1:  -11.467444053227585
glc__D_e_1:  -17.27279925329911
glc__D_e_1:  -15.615027937172282
glc__D_e_1:  -25.5872574532754
glc__D_e_1:  -0.4100373954873824
glc__D_e_1:  -10.874442679724774
glc__D_e_1:  -15.007072937628227
glc__D_e_1:  -7.08952911391976
glc__D_e_1:  -9.44650522355245
glc__D_e_1:  -9.017630036980826
glc__D_e_1:  -3.9440944802278

glc__D_e_1:  -1.2943246395835493
glc__D_e_1:  -1.7439080187271727
glc__D_e_1:  -1.6898289522623442
glc__D_e_1:  -0.5972566007430395
glc__D_e_1:  -0.902364783717974
glc__D_e_1:  -0.12138218474832452
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: [0.49885752915679404, 0], 17: [0.49885752915679404, 0], 18: [0.49885752915679404, 0], 19: [0.49885752915679404, 0], 20: [0.49885752915679404, 0], 21: [0.49885752915679404, 0], 22: [0.49885752915679404, 0], 23: [0.49885752915679404, 0], 24: [0.49885752915679404, 0], 25: [0.49885752915679404, 0], 26: [0.4988575

biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0, 0], 6: [0, 0], 7: [0, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0, 0], 13: [0, 0], 14: [0, 0], 15: [0, 0], 16: [0, 0], 17: [0, 0], 18: [0, 0], 19: [0, 0], 20: [0, 0], 21: [0, 0], 22: [0, 0], 23: [0, 0], 24: [0, 0], 25: [0, 0], 26: [0, 0], 27: [0, 0], 28: [0, 0], 29: [0, 0]}
strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 959.5230538295551, 336.5860805860806, 316.29835252466836, 420.4422065507592, 486.21788562314885, 513.3105194722593, 467.7152013974408, 413.7423070498547, 403.2117401543872, 463.2954270108147, 420.9083880961872, 379.73729475713577, 400.74133268002015, 444.4476744172621, 339.9219403547837, 322.28473043980375, 162.66754013698423, 256.73932245915, 198.15126050420167, 212.3181818181818, 275.0, 120.0, 75.0, 15.0, 0.0]
slip_r:  0.0
The initial distribution of microorganism is:  {}
The change process o

In [11]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 7,  3, 17, 10,  7,  4,  6,  6, 11,  1, 14,  1,  3,  9, 16,  5,  4,
       16,  5,  2, 17, 12, 15,  8,  3,  6, 16, 20,  9,  9]), 1: array([ 8,  6, 14, 10, 12, 20, 11, 15,  3,  8,  5, 19, 16, 16, 20,  9,  7,
       14,  6, 13,  3,  2,  9, 20,  4,  5, 18, 10, 20, 10])}
1
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

glc__D_e_1:  -19.482985925492805
glc__D_e_1:  -16.902920223363548
glc__D_e_1:  -24.91276444314996
glc__D_e_1:  -24.790122643616623
glc__D_e_1:  -28.03401447590104
glc__D_e_1:  -25.55250576438577
glc__D_e_1:  -13.373416718469631
glc__D_e_1:  -32.89003568754507
glc__D_e_1:  -0.11108206910396135
glc__D_e_1:  -7.610219386556524
glc__D_e_1:  -12.459822098179536
glc__D_e_1:  -0.11404978586898551
glc__D_e_1:  -16.730543018248927
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12

glc__D_e_1:  -9.097494910147736
glc__D_e_1:  -16.28974927363251
glc__D_e_1:  -6.534562155928781
glc__D_e_1:  -25.74501991392134
glc__D_e_1:  -15.821689507319869
glc__D_e_1:  -19.509910501160153
glc__D_e_1:  -27.740029171285606
glc__D_e_1:  -15.19114243683929
glc__D_e_1:  -10.418038817040113
glc__D_e_1:  -20.74091226597669
glc__D_e_1:  -25.56702704872241
glc__D_e_1:  -10.410436250964056
glc__D_e_1:  -18.200064301390782
glc__D_e_1:  -7.011289439449446
glc__D_e_1:  -17.221994241157176
glc__D_e_1:  -13.138214681223229
glc__D_e_1:  -4.540571458728484
glc__D_e_1:  -4.478233770841242
glc__D_e_1:  -7.58919271504186
glc__D_e_1:  -6.310447496404722
glc__D_e_1:  -6.000862915734504
glc__D_e_1:  -6.790354106074218
glc__D_e_1:  -2.3560415043389704
glc__D_e_1:  -5.574292963878582
glc__D_e_1:  -3.0379812166550657
glc__D_e_1:  -1.885741971354842
glc__D_e_1:  -4.656176034740767
glc__D_e_1:  -1.4680878718833534
glc__D_e_1:  -0.9808736774309079
glc__D_e_1:  -1.9568875523035447
biomass_strain:  {0: [0.4988

biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: [0.49885752915679404, 0], 17: [0.49885752915679404, 0], 18: [0.49885752915679404, 0], 19: [0.49885752915679404, 0], 20: [0.49885752915679404, 0], 21: [0.49885752915679404, 0], 22: [0.49885752915679404, 0], 23: [0.49885752915679404, 0], 24: [0.49885752915679404, 0], 25: [0.49885752915679404, 0], 26: [0.49885752915679404, 0], 27: [0, 0], 28: [0.49885752915679404, 0], 29: [0, 0]}
strain_number:  0 3.3333333333333335
strain_various:  0 2.4540216425741286
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_thr

biomass_strain:  {0: [0, 0], 1: [0, 0], 2: [0, 0], 3: [0, 0], 4: [0, 0], 5: [0, 0], 6: [0, 0], 7: [0, 0], 8: [0, 0], 9: [0, 0], 10: [0, 0], 11: [0, 0], 12: [0, 0], 13: [0, 0], 14: [0, 0], 15: [0, 0], 16: [0, 0], 17: [0, 0], 18: [0, 0], 19: [0, 0], 20: [0, 0], 21: [0, 0], 22: [0, 0], 23: [0, 0], 24: [0, 0], 25: [0, 0], 26: [0, 0], 27: [0, 0], 28: [0, 0], 29: [0, 0]}
strain_number:  0 0.0
strain_various:  0 0.0
strain_number:  1 0.0
strain_various:  1 0.0
fd_r_threshold:  [1.1, 1.1, 1.1, 1.1, 1.1, 670.508852989116, 354.1621744483586, 328.0135905760906, 412.32336413586415, 442.3214921043868, 460.47486789526266, 481.59049422799427, 456.2027134061938, 470.41870386145644, 467.0152200805351, 364.9135568846388, 421.5833107453989, 439.13381957579105, 457.10489443903504, 428.79389322613224, 347.0976893783811, 253.3031232596655, 212.19699736949815, 203.25808907988542, 198.8734491081375, 244.6258116883117, 245.41071428571428, 178.75, 140.0, 105.0, 45.0, 0.0]
slip_r:  0.0
The initial distribution o

In [12]:
#30
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.45)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([ 6,  1, 10, 15, 20,  1, 18,  3, 20,  1, 13,  9, 19,  5,  1, 11, 18,
        5,  5,  9,  4,  2,  5,  5, 12, 19,  7,  3, 18, 15]), 1: array([15,  4,  2, 20,  3,  8, 11, 18,  6,  1, 10, 15,  5, 12,  8,  3, 16,
       20, 19,  9, 14,  8, 17, 11, 20, 13, 16,  5, 14, 15])}
1
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

glc__D_e_1:  -7.44668316217383
glc__D_e_1:  -28.683004118730164
glc__D_e_1:  -0.4650906366809089
glc__D_e_1:  -17.655998919764723
glc__D_e_1:  -22.739900599058
glc__D_e_1:  -23.333598964324068
glc__D_e_1:  -22.267277107867184
glc__D_e_1:  -31.923456979478708
glc__D_e_1:  -16.89701331663746
glc__D_e_1:  -27.73868789135795
glc__D_e_1:  -20.68386918958234
glc__D_e_1:  -11.118846015531005
glc__D_e_1:  -10.249217688489871
glc__D_e_1:  -0.9441388004405589
glc__D_e_1:  -14.031616111637625
glc__D_e_1:  -0.16549708719645473
glc__D_e_1:  -0.49061897404268173
glc__D_e_1:  -13.23441117320032
glc__D_e_1:  -14.034444782561831
glc__D_e_1:  -0.8369998065531714
glc__D_e_1:  -7.871720640672127
glc__D_e_1:  -12.910101423797252
glc__D_e_1:  -0.5643291114867015
glc__D_e_1:  -6.974227805837595
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0.2857391744693639], 5: 

glc__D_e_1:  -25.03071904398803
glc__D_e_1:  -49.30309324180616
glc__D_e_1:  -0.46899983836075165
glc__D_e_1:  -18.58328055285285
glc__D_e_1:  -28.535409993991156
glc__D_e_1:  -0.31749651194190065
glc__D_e_1:  -4.44588447192789
glc__D_e_1:  -5.79450709819224
glc__D_e_1:  -3.9987177612143734
glc__D_e_1:  -4.967031757264121
glc__D_e_1:  -1.482868497615371
glc__D_e_1:  -2.214118789664513
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: [0.49885752915679404, 0], 17: [0.49885752915679404, 0], 18: [0.49885752915679404, 0], 19: [0.49885752915679404, 0], 20: 

glc__D_e_1:  -2.0023142615296585
glc__D_e_1:  -0.7457142671828699
glc__D_e_1:  -0.7208994682561016
glc__D_e_1:  -0.22455946791346504
glc__D_e_1:  -0.03654380506617905
glc__D_e_1:  -0.3266394615459368
glc__D_e_1:  -0.3728248731710566
glc__D_e_1:  -1.727737665883239
glc__D_e_1:  -0.9450500756031145
glc__D_e_1:  -0.22038579241564482
glc__D_e_1:  -2.724994414628962
glc__D_e_1:  -2.3479739421639043
glc__D_e_1:  -1.9352414430453508
glc__D_e_1:  -3.476770145907664
glc__D_e_1:  -4.2726054996800515
glc__D_e_1:  -2.848306258799254
glc__D_e_1:  -3.365543609650257
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 1

In [13]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.06, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([15,  4, 15,  4, 16,  7,  2, 20,  2, 15, 15,  7,  8, 12,  7, 18,  2,
       20,  6, 16, 13,  7, 20, 15, 11,  4, 12, 17,  4, 14]), 1: array([ 7,  3,  7, 13,  1,  7,  5,  2, 20,  7,  7, 12, 19,  5, 16,  2, 17,
       17, 14, 16, 10,  8, 13, 12,  4,  7, 13,  9, 17, 11])}
1
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [0.49885752915679404, 0.2857391744693639], 22: [0.49885752915679404, 0.285739174469363

glc__D_e_1:  -1.5580065631455724
glc__D_e_1:  -19.5052658836094
glc__D_e_1:  -14.978548614611293
glc__D_e_1:  -1.112429590170123
glc__D_e_1:  -0.049241853428512394
glc__D_e_1:  -14.354074188978007
glc__D_e_1:  -32.86936893826168
glc__D_e_1:  -0.759089368253079
glc__D_e_1:  -0.07475913026308434
glc__D_e_1:  -16.981325026465676
glc__D_e_1:  -35.08052545952584
glc__D_e_1:  -1.2669005621210232
glc__D_e_1:  -15.852639876512239
glc__D_e_1:  -27.532415899335035
glc__D_e_1:  -0.6518505141508051
glc__D_e_1:  -0.17169216429247358
glc__D_e_1:  -13.43583107558073
glc__D_e_1:  -39.115480295298546
glc__D_e_1:  -2.9298360220650004
glc__D_e_1:  -17.499796808067323
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915

glc__D_e_1:  -11.7577973736373
glc__D_e_1:  -9.625346516324885
glc__D_e_1:  -3.3694395644472426
glc__D_e_1:  -5.6998890302422796
glc__D_e_1:  -2.9572830852191623
glc__D_e_1:  -7.756318688620851
glc__D_e_1:  -2.7266031830908415
glc__D_e_1:  -4.150892363699607
glc__D_e_1:  -3.8013451792752813
glc__D_e_1:  -3.5348073306284453
glc__D_e_1:  -3.612600210368427
glc__D_e_1:  -9.521691475178228
glc__D_e_1:  -3.968720721570515
glc__D_e_1:  -9.075934277213717
glc__D_e_1:  -1.6506443363860948
glc__D_e_1:  -7.302505624529431
glc__D_e_1:  -4.945625476277777
glc__D_e_1:  -7.7507899948505985
glc__D_e_1:  -6.33616178090901
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915

glc__D_e_1:  -1.6367169169429205
glc__D_e_1:  -1.5710715679651255
glc__D_e_1:  -3.9747527516658514
glc__D_e_1:  -2.4226436148403137
glc__D_e_1:  -2.070412109732123
glc__D_e_1:  -2.134397937020431
glc__D_e_1:  -3.470627858268446
glc__D_e_1:  -1.955061405046575
glc__D_e_1:  -2.782357280418
glc__D_e_1:  -0.41581120041422515
glc__D_e_1:  -0.28813885427075747
glc__D_e_1:  -0.4228154633464979
glc__D_e_1:  -0.022951509908901047
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: [0.49885752915679404, 0], 17: [0, 0], 18: [0.49885752915679404, 0], 19: [0, 0], 20: [0.49885752915679

In [14]:
biomass_x_initial, biomass, number, various = time_space_state([model0, model1], ['BIOMASS_Ec_iML1515_core_75p37M', 'BIOMASS_SC5_notrace'], [0.7074, 0.28], [5, 30], [[0.13, 10, 1268], [200, 50, 3410]], public_metabolism, vmax, km, 1/15, 1, 0.1, 30, 0.1, 2.65)
print('The initial distribution of microorganism is: ',biomass_x_initial)
print('The change process of microorganism distribution is: ',biomass)
print('The mean numbers of microorganisms are: ', number)
print('The standard deviations of microorganisms are: ', various)

{0: array([18,  7,  8, 20, 12, 20,  1, 20, 18,  1, 10, 15, 11, 13, 13, 12, 19,
        6,  3, 11,  5,  9,  3,  2, 18, 20, 15, 19,  7,  8]), 1: array([ 2, 18, 11, 15, 17, 18,  7,  1,  2, 10,  5, 17,  3,  2,  4, 19, 11,
        1, 11,  8, 19, 10,  2, 16, 16, 15, 10,  4,  3,  9])}
1
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.2857391744693639], 6: [0.49885752915679404, 0.2857391744693639], 7: [0.49885752915679404, 0.2857391744693639], 8: [0.49885752915679404, 0.2857391744693639], 9: [0.49885752915679404, 0.2857391744693639], 10: [0.49885752915679404, 0.2857391744693639], 11: [0.49885752915679404, 0.2857391744693639], 12: [0.49885752915679404, 0.2857391744693639], 13: [0.49885752915679404, 0.2857391744693639], 14: [0.49885752915679404, 0.2857391744693639], 15: [0.49885752915679404, 0.2857391744693639], 16: [0.49885752915679404, 0.2857391744693639], 17: [0.49885752915679404, 0.2857391744693639], 18: [0.49885752915679404, 0.2857391744693639], 19: [0.49885752915679404, 0.2857391744693639], 20: [0.49885752915679404, 0.2857391744693639], 21: [

glc__D_e_1:  -2.154641454134296
glc__D_e_1:  -0.021963078236136302
glc__D_e_1:  -2.3588210347813905
glc__D_e_1:  -12.16116114909624
glc__D_e_1:  -0.0068519799168709206
glc__D_e_1:  -20.555844861033794
glc__D_e_1:  -28.98599896365098
glc__D_e_1:  -12.717906505918275
glc__D_e_1:  -37.30020775653942
glc__D_e_1:  -22.671048302266065
glc__D_e_1:  -29.77934783454253
glc__D_e_1:  -37.50691011975794
glc__D_e_1:  -24.23127828316219
glc__D_e_1:  -16.404458948004844
glc__D_e_1:  -23.47546546152054
glc__D_e_1:  -5.958792472323973
glc__D_e_1:  -11.184179733406683
glc__D_e_1:  -11.276496158482166
glc__D_e_1:  -11.264357021413755
glc__D_e_1:  -22.928331927375012
glc__D_e_1:  -38.07888118014173
glc__D_e_1:  -26.94381557953668
biomass_strain:  {0: [0.49885752915679404, 0.2857391744693639], 1: [0.49885752915679404, 0.2857391744693639], 2: [0.49885752915679404, 0.2857391744693639], 3: [0.49885752915679404, 0.2857391744693639], 4: [0.49885752915679404, 0.2857391744693639], 5: [0.49885752915679404, 0.28573

glc__D_e_1:  -7.391432328424245
glc__D_e_1:  -5.6328925011903515
biomass_strain:  {0: [0.49885752915679404, 0], 1: [0.49885752915679404, 0], 2: [0.49885752915679404, 0], 3: [0.49885752915679404, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0.49885752915679404, 0], 7: [0.49885752915679404, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: [0.49885752915679404, 0], 17: [0.49885752915679404, 0], 18: [0.49885752915679404, 0], 19: [0.49885752915679404, 0], 20: [0.49885752915679404, 0], 21: [0.49885752915679404, 0], 22: [0.49885752915679404, 0], 23: [0.49885752915679404, 0], 24: [0.49885752915679404, 0], 25: [0.49885752915679404, 0], 26: [0.49885752915679404, 0], 27: [0.49885752915679404, 0], 28: [0.49885752915679404, 0], 29: [0.49885752915679404, 0]}
strain_number:  0 13.7
stra

glc__D_e_1:  -0.39851283988992514
glc__D_e_1:  -0.08681750276075506
glc__D_e_1:  -0.03801691453976508
glc__D_e_1:  -0.143346166054374
glc__D_e_1:  -0.14237868730152314
glc__D_e_1:  -0.38236333631248354
glc__D_e_1:  -0.6593963040846442
glc__D_e_1:  -1.8823848820385662
glc__D_e_1:  -1.9495683283772405
glc__D_e_1:  -2.585712010948397
glc__D_e_1:  -0.5084693713596302
glc__D_e_1:  -0.7993729140290602
glc__D_e_1:  -2.2875566688819093
glc__D_e_1:  -2.850652642797252
glc__D_e_1:  -2.2673893148281543
glc__D_e_1:  -0.6235028501089371
glc__D_e_1:  -0.03654155108919044
glc__D_e_1:  -0.01940011858714663
biomass_strain:  {0: [0, 0], 1: [0.49885752915679404, 0], 2: [0, 0], 3: [0, 0], 4: [0.49885752915679404, 0], 5: [0.49885752915679404, 0], 6: [0, 0], 7: [0, 0], 8: [0.49885752915679404, 0], 9: [0.49885752915679404, 0], 10: [0.49885752915679404, 0], 11: [0.49885752915679404, 0], 12: [0.49885752915679404, 0], 13: [0.49885752915679404, 0], 14: [0.49885752915679404, 0], 15: [0.49885752915679404, 0], 16: 